In [10]:
import caffe
import numpy as np
from numpy import array
from numpy.random import normal
from matplotlib import pyplot
import os, sys
import struct
prototxt = '../Darknet19/Darknet19.prototxt'
caffemodel = '../Darknet19/Darknet19.caffemodel'
TEST_ITER = 375
caffe.set_device(0)  
caffe.set_mode_gpu()


In [11]:
def test_accuracy(net, test_iter):
    loss = np.zeros(1)
    top1 = np.zeros(1)
    top5 = np.zeros(1)
    test_loss = 0
    top1_accuracy = 0
    top5_accuracy = 0
    for test_it in range(test_iter):
        net.forward()
        test_loss += net.blobs['loss'].data
        top1_accuracy += net.blobs['accuracy'].data  
        top5_accuracy += net.blobs['accuracy-top5'].data
    loss[0] = test_loss / test_iter 
    top1[0] = top1_accuracy / test_iter
    top5[0] = top5_accuracy / test_iter
    print 'Loss: ', loss[0], ', Accuracy Top1:', top1[0], ', Accuracy Top5:', top5[0]
    log.writelines('Loss: '+ str(loss[0]) + ', Accuracy Top1:' + str(top1[0]) + ', Accuracy Top5:'+ str(top5[0])+'\n')
    return loss[0], top1[0], top5[0]

def prune_fm(prototxt, caffemodel, layer, iteration):#剪枝哪一层，这一层有几个fm，每次简直测试跑多少iteration
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)

    zeros = np.zeros(net.params[layer][0].data[0].shape)#用于剪枝
    loss = np.zeros(net.blobs[layer].channels)
    top1 = np.zeros(net.blobs[layer].channels)
    top5 = np.zeros(net.blobs[layer].channels)
    
    for i in range(net.blobs[layer].channels):
        net.copy_from(caffemodel)
        net.params[layer][0].data[i] = zeros
        print 'pruning', layer, 'feature map',i, '...'
        log.writelines('pruning ' + layer + ' feature map ' + str(i) + '...\n')
        loss[i],top1[i],top5[i] = test_accuracy(net, iteration)
    return loss,top1,top5

def test_pruned_fm(prototxt, caffemodel, layer, iteration, prune_rate):
    
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)

    zeros=np.zeros(net.params[layer][0].data[0].shape)
    loss = np.zeros(net.blobs[layer].channels)
    top1 = np.zeros(net.blobs[layer].channels)
    top5 = np.zeros(net.blobs[layer].channels)
    
    prune_cnt = int(prune_rate*net.blobs[layer].channels)
    
    [loss,top1,top5]=prune_fm(prototxt, caffemodel, layer, iteration)
    
    print 'cut unimportant features...'
    log.writelines('cut unimportant features... \n')
    #重要程度越低越靠前
    loss_index = np.argsort(loss)
    top1_index = np.argsort(-top1)
    top5_index = np.argsort(-top5)
    
    #根据top1做剪枝
    print 'prune basing on top1...'
    log.writelines('prune basing on top1... \n')
    print 'fm to be pruned: ', top1_index[:prune_cnt]
    log.writelines('fm to be pruned: '+ str(top1_index[:prune_cnt])+'\n')           
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)
    for i in range(prune_cnt):
        net.params[layer][0].data[top1_index[i]]=zeros
    test_accuracy(net, TEST_ITER)
    
    #根据loss做剪枝
    print 'prune basing on loss...'
    log.writelines('prune basing on loss... \n')
    print 'fm to be pruned:', loss_index[:prune_cnt]
    log.writelines('fm to be pruned: '+ str(loss_index[:prune_cnt])+'\n')       
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)
    for i in range(prune_cnt):
        net.params[layer][0].data[loss_index[i]]=zeros
    test_accuracy(net, TEST_ITER)
    
    print 'cut important features...'
    log.writelines('prune basing on top1... \n')
    #重要程度越低越靠前
    loss_index = np.argsort(-loss)
    top1_index = np.argsort(top1)
    top5_index = np.argsort(top5)
    
    #根据top1做剪枝
    print 'prune basing on top1...'
    log.writelines('prune basing on top1... \n')
    print 'fm to be pruned:', top1_index[:prune_cnt]
    log.writelines('fm to be pruned: '+ str(top1_index[:prune_cnt])+'\n') 
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)
    for i in range(prune_cnt):
        net.params[layer][0].data[top1_index[i]]=zeros
    test_accuracy(net, TEST_ITER)
    
    #根据loss做剪枝
    print 'prune basing on loss...'
    log.writelines('prune basing on top1... \n')
    print 'fm to be pruned:', loss_index[:prune_cnt]
    log.writelines('fm to be pruned: '+ str(loss_index[:prune_cnt])+'\n') 
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)
    for i in range(prune_cnt):
        net.params[layer][0].data[loss_index[i]]=zeros
    test_accuracy(net, TEST_ITER)

In [13]:
log=open("Darkent1.log",'a+') 
test_pruned_fm(prototxt, caffemodel, 'conv1', 100, 0.375)
log.close()

pruning conv1 feature map 0 ...
Loss:  0.7315416458249092 , Accuracy Top1: 0.78625 , Accuracy Top5: 0.9640625
pruning conv1 feature map 1 ...
Loss:  0.6900710919499398 , Accuracy Top1: 0.801875 , Accuracy Top5: 0.9659375
pruning conv1 feature map 2 ...
Loss:  0.7621063500642776 , Accuracy Top1: 0.7765625 , Accuracy Top5: 0.969375
pruning conv1 feature map 3 ...
Loss:  0.7103455550968647 , Accuracy Top1: 0.7934375 , Accuracy Top5: 0.9634375
pruning conv1 feature map 4 ...
Loss:  0.7352225121855736 , Accuracy Top1: 0.7853125 , Accuracy Top5: 0.9615625
pruning conv1 feature map 5 ...
Loss:  0.7105121424794197 , Accuracy Top1: 0.78875 , Accuracy Top5: 0.9659375
pruning conv1 feature map 6 ...
Loss:  0.7288758413493633 , Accuracy Top1: 0.7896875 , Accuracy Top5: 0.9678125
pruning conv1 feature map 7 ...
Loss:  0.8504715472459793 , Accuracy Top1: 0.7578125 , Accuracy Top5: 0.95625
pruning conv1 feature map 8 ...
Loss:  0.7092348185181617 , Accuracy Top1: 0.790625 , Accuracy Top5: 0.966875
pr

In [15]:
test_pruned_fm(prototxt, caffemodel, 'conv2', 200, 0.375)

pruning conv2 feature map 0 ...
Loss:  2.007403064966202 Accuracy Top1: 0.4971875 Accuracy Top5: 0.86734375
pruning conv2 feature map 1 ...
Loss:  1.9647955727577209 Accuracy Top1: 0.50859375 Accuracy Top5: 0.8740625
pruning conv2 feature map 2 ...
Loss:  2.045591720342636 Accuracy Top1: 0.49140625 Accuracy Top5: 0.86109375
pruning conv2 feature map 3 ...
Loss:  1.980151275396347 Accuracy Top1: 0.5053125 Accuracy Top5: 0.87375
pruning conv2 feature map 4 ...
Loss:  1.9701991486549377 Accuracy Top1: 0.50453125 Accuracy Top5: 0.86734375
pruning conv2 feature map 5 ...
Loss:  1.9839280745387078 Accuracy Top1: 0.50671875 Accuracy Top5: 0.87265625
pruning conv2 feature map 6 ...
Loss:  2.0100025066733362 Accuracy Top1: 0.4975 Accuracy Top5: 0.86765625
pruning conv2 feature map 7 ...
Loss:  2.031570007801056 Accuracy Top1: 0.50015625 Accuracy Top5: 0.86421875
pruning conv2 feature map 8 ...
Loss:  1.985551704764366 Accuracy Top1: 0.50109375 Accuracy Top5: 0.8646875
pruning conv2 feature map 

Loss:  2.028389647603035 Accuracy Top1: 0.4940625 Accuracy Top5: 0.8646875
pruning conv3 feature map 5 ...


KeyboardInterrupt: 

In [13]:
log=open("Darkent.log",'a+') 
test_pruned_fm(prototxt, caffemodel, 'conv5', 100, 0.375)
log.close()

pruning conv5 feature map 0 ...
Loss:  2.0345578265190123 , Accuracy Top1: 0.495625 , Accuracy Top5: 0.8628125
pruning conv5 feature map 1 ...
Loss:  2.009782559275627 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.8653125
pruning conv5 feature map 2 ...
Loss:  1.9951766312122345 , Accuracy Top1: 0.5015625 , Accuracy Top5: 0.874375
pruning conv5 feature map 3 ...
Loss:  1.9234566962718964 , Accuracy Top1: 0.5178125 , Accuracy Top5: 0.8753125
pruning conv5 feature map 4 ...
Loss:  2.063266236782074 , Accuracy Top1: 0.4925 , Accuracy Top5: 0.8596875
pruning conv5 feature map 5 ...
Loss:  2.0169641560316087 , Accuracy Top1: 0.495625 , Accuracy Top5: 0.861875
pruning conv5 feature map 6 ...
Loss:  1.9100711858272552 , Accuracy Top1: 0.5196875 , Accuracy Top5: 0.8828125
pruning conv5 feature map 7 ...
Loss:  1.9678546351194381 , Accuracy Top1: 0.5078125 , Accuracy Top5: 0.873125
pruning conv5 feature map 8 ...
Loss:  2.036070218086243 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.86375
pruning

Loss:  1.9644050616025925 , Accuracy Top1: 0.5134375 , Accuracy Top5: 0.8753125
pruning conv5 feature map 75 ...
Loss:  2.0235864555835725 , Accuracy Top1: 0.4984375 , Accuracy Top5: 0.861875
pruning conv5 feature map 76 ...
Loss:  2.0103450876474382 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.8675
pruning conv5 feature map 77 ...
Loss:  1.9903964912891388 , Accuracy Top1: 0.501875 , Accuracy Top5: 0.8753125
pruning conv5 feature map 78 ...
Loss:  1.9158415079116822 , Accuracy Top1: 0.519375 , Accuracy Top5: 0.8759375
pruning conv5 feature map 79 ...
Loss:  2.0569167160987853 , Accuracy Top1: 0.4921875 , Accuracy Top5: 0.8603125
pruning conv5 feature map 80 ...
Loss:  2.0135757887363432 , Accuracy Top1: 0.4965625 , Accuracy Top5: 0.8640625
pruning conv5 feature map 81 ...
Loss:  1.9624485087394714 , Accuracy Top1: 0.50875 , Accuracy Top5: 0.875
pruning conv5 feature map 82 ...
Loss:  1.9616443157196044 , Accuracy Top1: 0.508125 , Accuracy Top5: 0.8740625
pruning conv5 feature map 83 ...

In [5]:
test_pruned_fm(prototxt, caffemodel, 'conv6', 50, 0.375)
test_pruned_fm(prototxt, caffemodel, 'conv7', 50, 0.375)
test_pruned_fm(prototxt, caffemodel, 'conv8', 50, 0.375)


pruning conv6 feature map 0 ...
Loss:  2.0404905390739443 Accuracy Top1: 0.494375 Accuracy Top5: 0.86375
pruning conv6 feature map 1 ...
Loss:  2.018864951133728 Accuracy Top1: 0.500625 Accuracy Top5: 0.863125
pruning conv6 feature map 2 ...
Loss:  2.0445695662498475 Accuracy Top1: 0.49625 Accuracy Top5: 0.864375
pruning conv6 feature map 3 ...
Loss:  2.0246170282363893 Accuracy Top1: 0.486875 Accuracy Top5: 0.865
pruning conv6 feature map 4 ...
Loss:  1.8835715675354003 Accuracy Top1: 0.5175 Accuracy Top5: 0.8775
pruning conv6 feature map 5 ...
Loss:  2.1041370034217834 Accuracy Top1: 0.484375 Accuracy Top5: 0.871875
pruning conv6 feature map 6 ...
Loss:  1.9189349269866944 Accuracy Top1: 0.52 Accuracy Top5: 0.87625
pruning conv6 feature map 7 ...
Loss:  1.9645087146759033 Accuracy Top1: 0.51375 Accuracy Top5: 0.870625
pruning conv6 feature map 8 ...
Loss:  2.0455514717102052 Accuracy Top1: 0.4975 Accuracy Top5: 0.8625
pruning conv6 feature map 9 ...
Loss:  2.068847804069519 Accuracy 

Loss:  2.0111893701553343 Accuracy Top1: 0.49375 Accuracy Top5: 0.864375
pruning conv6 feature map 79 ...
Loss:  1.895858838558197 Accuracy Top1: 0.515625 Accuracy Top5: 0.87625
pruning conv6 feature map 80 ...
Loss:  2.079520206451416 Accuracy Top1: 0.49 Accuracy Top5: 0.874375
pruning conv6 feature map 81 ...
Loss:  1.9453647637367248 Accuracy Top1: 0.515625 Accuracy Top5: 0.876875
pruning conv6 feature map 82 ...
Loss:  1.9327024483680726 Accuracy Top1: 0.516875 Accuracy Top5: 0.874375
pruning conv6 feature map 83 ...
Loss:  2.0416580629348755 Accuracy Top1: 0.501875 Accuracy Top5: 0.863125
pruning conv6 feature map 84 ...
Loss:  2.0316171050071716 Accuracy Top1: 0.495625 Accuracy Top5: 0.863125
pruning conv6 feature map 85 ...
Loss:  2.034164559841156 Accuracy Top1: 0.495625 Accuracy Top5: 0.85375
pruning conv6 feature map 86 ...
Loss:  1.9792770612239838 Accuracy Top1: 0.495625 Accuracy Top5: 0.866875
pruning conv6 feature map 87 ...
Loss:  1.922912380695343 Accuracy Top1: 0.50625

Loss:  1.9271670258045197 Accuracy Top1: 0.5175 Accuracy Top5: 0.87875
pruning conv6 feature map 157 ...
Loss:  1.9391193151474 Accuracy Top1: 0.515 Accuracy Top5: 0.87
pruning conv6 feature map 158 ...
Loss:  2.0351654744148253 Accuracy Top1: 0.5025 Accuracy Top5: 0.864375
pruning conv6 feature map 159 ...
Loss:  2.0625829982757566 Accuracy Top1: 0.491875 Accuracy Top5: 0.863125
pruning conv6 feature map 160 ...
Loss:  2.001220338344574 Accuracy Top1: 0.498125 Accuracy Top5: 0.861875
pruning conv6 feature map 161 ...
Loss:  2.0006676149368285 Accuracy Top1: 0.49625 Accuracy Top5: 0.865
pruning conv6 feature map 162 ...
Loss:  1.915450038909912 Accuracy Top1: 0.51 Accuracy Top5: 0.881875
pruning conv6 feature map 163 ...
Loss:  1.9952025079727174 Accuracy Top1: 0.511875 Accuracy Top5: 0.871875
pruning conv6 feature map 164 ...
Loss:  1.903899986743927 Accuracy Top1: 0.51875 Accuracy Top5: 0.880625
pruning conv6 feature map 165 ...
Loss:  2.037961976528168 Accuracy Top1: 0.49625 Accurac

Loss:  2.0547543740272523 Accuracy Top1: 0.494375 Accuracy Top5: 0.8625
pruning conv6 feature map 235 ...
Loss:  2.031303143501282 Accuracy Top1: 0.494375 Accuracy Top5: 0.860625
pruning conv6 feature map 236 ...
Loss:  2.024426565170288 Accuracy Top1: 0.490625 Accuracy Top5: 0.86375
pruning conv6 feature map 237 ...
Loss:  1.9382713174819945 Accuracy Top1: 0.506875 Accuracy Top5: 0.880625
pruning conv6 feature map 238 ...
Loss:  1.973586437702179 Accuracy Top1: 0.514375 Accuracy Top5: 0.86875
pruning conv6 feature map 239 ...
Loss:  1.8949015045166016 Accuracy Top1: 0.52 Accuracy Top5: 0.881875
pruning conv6 feature map 240 ...
Loss:  2.035516004562378 Accuracy Top1: 0.496875 Accuracy Top5: 0.868125
pruning conv6 feature map 241 ...
Loss:  2.0448019313812256 Accuracy Top1: 0.495 Accuracy Top5: 0.86
pruning conv6 feature map 242 ...
Loss:  2.0349907946586607 Accuracy Top1: 0.49875 Accuracy Top5: 0.861875
pruning conv6 feature map 243 ...
Loss:  2.0091764307022095 Accuracy Top1: 0.48875

Loss:  1.9481344509124756 Accuracy Top1: 0.51625 Accuracy Top5: 0.870625
pruning conv7 feature map 38 ...
Loss:  2.020791492462158 Accuracy Top1: 0.499375 Accuracy Top5: 0.86375
pruning conv7 feature map 39 ...
Loss:  2.0568909358978273 Accuracy Top1: 0.49125 Accuracy Top5: 0.8625
pruning conv7 feature map 40 ...
Loss:  2.0187472987174986 Accuracy Top1: 0.4975 Accuracy Top5: 0.8625
pruning conv7 feature map 41 ...
Loss:  2.0469347643852234 Accuracy Top1: 0.495 Accuracy Top5: 0.8625
pruning conv7 feature map 42 ...
Loss:  1.929531888961792 Accuracy Top1: 0.509375 Accuracy Top5: 0.88
pruning conv7 feature map 43 ...
Loss:  2.0363932061195373 Accuracy Top1: 0.508125 Accuracy Top5: 0.866875
pruning conv7 feature map 44 ...
Loss:  1.8669576692581176 Accuracy Top1: 0.5225 Accuracy Top5: 0.88125
pruning conv7 feature map 45 ...
Loss:  2.0289346837997435 Accuracy Top1: 0.49625 Accuracy Top5: 0.86625
pruning conv7 feature map 46 ...
Loss:  2.039497883319855 Accuracy Top1: 0.4975 Accuracy Top5: 

Loss:  1.9947414803504944 Accuracy Top1: 0.496875 Accuracy Top5: 0.864375
pruning conv7 feature map 117 ...
Loss:  1.918305718898773 Accuracy Top1: 0.505 Accuracy Top5: 0.88125
pruning conv7 feature map 118 ...
Loss:  2.050346269607544 Accuracy Top1: 0.504375 Accuracy Top5: 0.865625
pruning conv7 feature map 119 ...
Loss:  1.860515956878662 Accuracy Top1: 0.525625 Accuracy Top5: 0.883125
pruning conv7 feature map 120 ...
Loss:  2.038723428249359 Accuracy Top1: 0.496875 Accuracy Top5: 0.86625
pruning conv7 feature map 121 ...
Loss:  1.9777762722969054 Accuracy Top1: 0.501875 Accuracy Top5: 0.86625
pruning conv7 feature map 122 ...
Loss:  2.0336101603507997 Accuracy Top1: 0.4975 Accuracy Top5: 0.86375
pruning conv7 feature map 123 ...
Loss:  2.01785808801651 Accuracy Top1: 0.4875 Accuracy Top5: 0.86625
pruning conv7 feature map 124 ...
Loss:  1.8631132411956788 Accuracy Top1: 0.52 Accuracy Top5: 0.874375
pruning conv7 feature map 125 ...
Loss:  2.1265460205078126 Accuracy Top1: 0.48 Accu

Loss:  2.073324534893036 Accuracy Top1: 0.488125 Accuracy Top5: 0.86125
pruning conv8 feature map 55 ...
Loss:  2.0154984259605406 Accuracy Top1: 0.498125 Accuracy Top5: 0.859375
pruning conv8 feature map 56 ...
Loss:  2.001068091392517 Accuracy Top1: 0.494375 Accuracy Top5: 0.86625
pruning conv8 feature map 57 ...
Loss:  1.9593664813041687 Accuracy Top1: 0.501875 Accuracy Top5: 0.878125
pruning conv8 feature map 58 ...
Loss:  2.0346303749084473 Accuracy Top1: 0.505 Accuracy Top5: 0.86625
pruning conv8 feature map 59 ...
Loss:  1.8771904218196869 Accuracy Top1: 0.52 Accuracy Top5: 0.883125
pruning conv8 feature map 60 ...
Loss:  2.035714530944824 Accuracy Top1: 0.49625 Accuracy Top5: 0.86625
pruning conv8 feature map 61 ...
Loss:  2.0243282651901247 Accuracy Top1: 0.499375 Accuracy Top5: 0.861875
pruning conv8 feature map 62 ...
Loss:  2.052863349914551 Accuracy Top1: 0.495 Accuracy Top5: 0.863125
pruning conv8 feature map 63 ...
Loss:  2.0257719373703003 Accuracy Top1: 0.48375 Accurac

Loss:  1.9288449811935424 Accuracy Top1: 0.509375 Accuracy Top5: 0.88125
pruning conv8 feature map 133 ...
Loss:  1.9892040777206421 Accuracy Top1: 0.51125 Accuracy Top5: 0.8725
pruning conv8 feature map 134 ...
Loss:  1.903970446586609 Accuracy Top1: 0.51625 Accuracy Top5: 0.880625
pruning conv8 feature map 135 ...
Loss:  1.9996105933189392 Accuracy Top1: 0.50375 Accuracy Top5: 0.869375
pruning conv8 feature map 136 ...
Loss:  2.0616500997543334 Accuracy Top1: 0.49 Accuracy Top5: 0.85875
pruning conv8 feature map 137 ...
Loss:  2.039880576133728 Accuracy Top1: 0.49625 Accuracy Top5: 0.865625
pruning conv8 feature map 138 ...
Loss:  2.0244516909122465 Accuracy Top1: 0.488125 Accuracy Top5: 0.866875
pruning conv8 feature map 139 ...
Loss:  1.891148874759674 Accuracy Top1: 0.513125 Accuracy Top5: 0.878125
pruning conv8 feature map 140 ...
Loss:  2.1011008048057556 Accuracy Top1: 0.484375 Accuracy Top5: 0.8725
pruning conv8 feature map 141 ...
Loss:  1.9106843388080597 Accuracy Top1: 0.52

Loss:  2.0431022000312806 Accuracy Top1: 0.495 Accuracy Top5: 0.866875
pruning conv8 feature map 211 ...
Loss:  2.002089240550995 Accuracy Top1: 0.49875 Accuracy Top5: 0.864375
pruning conv8 feature map 212 ...
Loss:  2.02299729347229 Accuracy Top1: 0.498125 Accuracy Top5: 0.86375
pruning conv8 feature map 213 ...
Loss:  2.03475177526474 Accuracy Top1: 0.48875 Accuracy Top5: 0.865
pruning conv8 feature map 214 ...
Loss:  1.8932553148269653 Accuracy Top1: 0.515625 Accuracy Top5: 0.876875
pruning conv8 feature map 215 ...
Loss:  2.0921590924263 Accuracy Top1: 0.4875 Accuracy Top5: 0.870625
pruning conv8 feature map 216 ...
Loss:  1.929418567419052 Accuracy Top1: 0.518125 Accuracy Top5: 0.8775
pruning conv8 feature map 217 ...
Loss:  1.950508623123169 Accuracy Top1: 0.515 Accuracy Top5: 0.87125
pruning conv8 feature map 218 ...
Loss:  2.04551922082901 Accuracy Top1: 0.496875 Accuracy Top5: 0.863125
pruning conv8 feature map 219 ...
Loss:  2.0786232590675353 Accuracy Top1: 0.486875 Accurac

In [5]:
test_pruned_fm(prototxt, caffemodel, 'conv6', 50, 0.375)
test_pruned_fm(prototxt, caffemodel, 'conv7', 50, 0.375)
test_pruned_fm(prototxt, caffemodel, 'conv8', 50, 0.375)


pruning conv6 feature map 0 ...
Loss:  2.0404905390739443 Accuracy Top1: 0.494375 Accuracy Top5: 0.86375
pruning conv6 feature map 1 ...
Loss:  2.018864951133728 Accuracy Top1: 0.500625 Accuracy Top5: 0.863125
pruning conv6 feature map 2 ...
Loss:  2.0445695662498475 Accuracy Top1: 0.49625 Accuracy Top5: 0.864375
pruning conv6 feature map 3 ...
Loss:  2.0246170282363893 Accuracy Top1: 0.486875 Accuracy Top5: 0.865
pruning conv6 feature map 4 ...
Loss:  1.8835715675354003 Accuracy Top1: 0.5175 Accuracy Top5: 0.8775
pruning conv6 feature map 5 ...
Loss:  2.1041370034217834 Accuracy Top1: 0.484375 Accuracy Top5: 0.871875
pruning conv6 feature map 6 ...
Loss:  1.9189349269866944 Accuracy Top1: 0.52 Accuracy Top5: 0.87625
pruning conv6 feature map 7 ...
Loss:  1.9645087146759033 Accuracy Top1: 0.51375 Accuracy Top5: 0.870625
pruning conv6 feature map 8 ...
Loss:  2.0455514717102052 Accuracy Top1: 0.4975 Accuracy Top5: 0.8625
pruning conv6 feature map 9 ...
Loss:  2.068847804069519 Accuracy 

Loss:  2.0111893701553343 Accuracy Top1: 0.49375 Accuracy Top5: 0.864375
pruning conv6 feature map 79 ...
Loss:  1.895858838558197 Accuracy Top1: 0.515625 Accuracy Top5: 0.87625
pruning conv6 feature map 80 ...
Loss:  2.079520206451416 Accuracy Top1: 0.49 Accuracy Top5: 0.874375
pruning conv6 feature map 81 ...
Loss:  1.9453647637367248 Accuracy Top1: 0.515625 Accuracy Top5: 0.876875
pruning conv6 feature map 82 ...
Loss:  1.9327024483680726 Accuracy Top1: 0.516875 Accuracy Top5: 0.874375
pruning conv6 feature map 83 ...
Loss:  2.0416580629348755 Accuracy Top1: 0.501875 Accuracy Top5: 0.863125
pruning conv6 feature map 84 ...
Loss:  2.0316171050071716 Accuracy Top1: 0.495625 Accuracy Top5: 0.863125
pruning conv6 feature map 85 ...
Loss:  2.034164559841156 Accuracy Top1: 0.495625 Accuracy Top5: 0.85375
pruning conv6 feature map 86 ...
Loss:  1.9792770612239838 Accuracy Top1: 0.495625 Accuracy Top5: 0.866875
pruning conv6 feature map 87 ...
Loss:  1.922912380695343 Accuracy Top1: 0.50625

Loss:  1.9271670258045197 Accuracy Top1: 0.5175 Accuracy Top5: 0.87875
pruning conv6 feature map 157 ...
Loss:  1.9391193151474 Accuracy Top1: 0.515 Accuracy Top5: 0.87
pruning conv6 feature map 158 ...
Loss:  2.0351654744148253 Accuracy Top1: 0.5025 Accuracy Top5: 0.864375
pruning conv6 feature map 159 ...
Loss:  2.0625829982757566 Accuracy Top1: 0.491875 Accuracy Top5: 0.863125
pruning conv6 feature map 160 ...
Loss:  2.001220338344574 Accuracy Top1: 0.498125 Accuracy Top5: 0.861875
pruning conv6 feature map 161 ...
Loss:  2.0006676149368285 Accuracy Top1: 0.49625 Accuracy Top5: 0.865
pruning conv6 feature map 162 ...
Loss:  1.915450038909912 Accuracy Top1: 0.51 Accuracy Top5: 0.881875
pruning conv6 feature map 163 ...
Loss:  1.9952025079727174 Accuracy Top1: 0.511875 Accuracy Top5: 0.871875
pruning conv6 feature map 164 ...
Loss:  1.903899986743927 Accuracy Top1: 0.51875 Accuracy Top5: 0.880625
pruning conv6 feature map 165 ...
Loss:  2.037961976528168 Accuracy Top1: 0.49625 Accurac

Loss:  2.0547543740272523 Accuracy Top1: 0.494375 Accuracy Top5: 0.8625
pruning conv6 feature map 235 ...
Loss:  2.031303143501282 Accuracy Top1: 0.494375 Accuracy Top5: 0.860625
pruning conv6 feature map 236 ...
Loss:  2.024426565170288 Accuracy Top1: 0.490625 Accuracy Top5: 0.86375
pruning conv6 feature map 237 ...
Loss:  1.9382713174819945 Accuracy Top1: 0.506875 Accuracy Top5: 0.880625
pruning conv6 feature map 238 ...
Loss:  1.973586437702179 Accuracy Top1: 0.514375 Accuracy Top5: 0.86875
pruning conv6 feature map 239 ...
Loss:  1.8949015045166016 Accuracy Top1: 0.52 Accuracy Top5: 0.881875
pruning conv6 feature map 240 ...
Loss:  2.035516004562378 Accuracy Top1: 0.496875 Accuracy Top5: 0.868125
pruning conv6 feature map 241 ...
Loss:  2.0448019313812256 Accuracy Top1: 0.495 Accuracy Top5: 0.86
pruning conv6 feature map 242 ...
Loss:  2.0349907946586607 Accuracy Top1: 0.49875 Accuracy Top5: 0.861875
pruning conv6 feature map 243 ...
Loss:  2.0091764307022095 Accuracy Top1: 0.48875

Loss:  1.9481344509124756 Accuracy Top1: 0.51625 Accuracy Top5: 0.870625
pruning conv7 feature map 38 ...
Loss:  2.020791492462158 Accuracy Top1: 0.499375 Accuracy Top5: 0.86375
pruning conv7 feature map 39 ...
Loss:  2.0568909358978273 Accuracy Top1: 0.49125 Accuracy Top5: 0.8625
pruning conv7 feature map 40 ...
Loss:  2.0187472987174986 Accuracy Top1: 0.4975 Accuracy Top5: 0.8625
pruning conv7 feature map 41 ...
Loss:  2.0469347643852234 Accuracy Top1: 0.495 Accuracy Top5: 0.8625
pruning conv7 feature map 42 ...
Loss:  1.929531888961792 Accuracy Top1: 0.509375 Accuracy Top5: 0.88
pruning conv7 feature map 43 ...
Loss:  2.0363932061195373 Accuracy Top1: 0.508125 Accuracy Top5: 0.866875
pruning conv7 feature map 44 ...
Loss:  1.8669576692581176 Accuracy Top1: 0.5225 Accuracy Top5: 0.88125
pruning conv7 feature map 45 ...
Loss:  2.0289346837997435 Accuracy Top1: 0.49625 Accuracy Top5: 0.86625
pruning conv7 feature map 46 ...
Loss:  2.039497883319855 Accuracy Top1: 0.4975 Accuracy Top5: 

Loss:  1.9947414803504944 Accuracy Top1: 0.496875 Accuracy Top5: 0.864375
pruning conv7 feature map 117 ...
Loss:  1.918305718898773 Accuracy Top1: 0.505 Accuracy Top5: 0.88125
pruning conv7 feature map 118 ...
Loss:  2.050346269607544 Accuracy Top1: 0.504375 Accuracy Top5: 0.865625
pruning conv7 feature map 119 ...
Loss:  1.860515956878662 Accuracy Top1: 0.525625 Accuracy Top5: 0.883125
pruning conv7 feature map 120 ...
Loss:  2.038723428249359 Accuracy Top1: 0.496875 Accuracy Top5: 0.86625
pruning conv7 feature map 121 ...
Loss:  1.9777762722969054 Accuracy Top1: 0.501875 Accuracy Top5: 0.86625
pruning conv7 feature map 122 ...
Loss:  2.0336101603507997 Accuracy Top1: 0.4975 Accuracy Top5: 0.86375
pruning conv7 feature map 123 ...
Loss:  2.01785808801651 Accuracy Top1: 0.4875 Accuracy Top5: 0.86625
pruning conv7 feature map 124 ...
Loss:  1.8631132411956788 Accuracy Top1: 0.52 Accuracy Top5: 0.874375
pruning conv7 feature map 125 ...
Loss:  2.1265460205078126 Accuracy Top1: 0.48 Accu

Loss:  2.073324534893036 Accuracy Top1: 0.488125 Accuracy Top5: 0.86125
pruning conv8 feature map 55 ...
Loss:  2.0154984259605406 Accuracy Top1: 0.498125 Accuracy Top5: 0.859375
pruning conv8 feature map 56 ...
Loss:  2.001068091392517 Accuracy Top1: 0.494375 Accuracy Top5: 0.86625
pruning conv8 feature map 57 ...
Loss:  1.9593664813041687 Accuracy Top1: 0.501875 Accuracy Top5: 0.878125
pruning conv8 feature map 58 ...
Loss:  2.0346303749084473 Accuracy Top1: 0.505 Accuracy Top5: 0.86625
pruning conv8 feature map 59 ...
Loss:  1.8771904218196869 Accuracy Top1: 0.52 Accuracy Top5: 0.883125
pruning conv8 feature map 60 ...
Loss:  2.035714530944824 Accuracy Top1: 0.49625 Accuracy Top5: 0.86625
pruning conv8 feature map 61 ...
Loss:  2.0243282651901247 Accuracy Top1: 0.499375 Accuracy Top5: 0.861875
pruning conv8 feature map 62 ...
Loss:  2.052863349914551 Accuracy Top1: 0.495 Accuracy Top5: 0.863125
pruning conv8 feature map 63 ...
Loss:  2.0257719373703003 Accuracy Top1: 0.48375 Accurac

Loss:  1.9288449811935424 Accuracy Top1: 0.509375 Accuracy Top5: 0.88125
pruning conv8 feature map 133 ...
Loss:  1.9892040777206421 Accuracy Top1: 0.51125 Accuracy Top5: 0.8725
pruning conv8 feature map 134 ...
Loss:  1.903970446586609 Accuracy Top1: 0.51625 Accuracy Top5: 0.880625
pruning conv8 feature map 135 ...
Loss:  1.9996105933189392 Accuracy Top1: 0.50375 Accuracy Top5: 0.869375
pruning conv8 feature map 136 ...
Loss:  2.0616500997543334 Accuracy Top1: 0.49 Accuracy Top5: 0.85875
pruning conv8 feature map 137 ...
Loss:  2.039880576133728 Accuracy Top1: 0.49625 Accuracy Top5: 0.865625
pruning conv8 feature map 138 ...
Loss:  2.0244516909122465 Accuracy Top1: 0.488125 Accuracy Top5: 0.866875
pruning conv8 feature map 139 ...
Loss:  1.891148874759674 Accuracy Top1: 0.513125 Accuracy Top5: 0.878125
pruning conv8 feature map 140 ...
Loss:  2.1011008048057556 Accuracy Top1: 0.484375 Accuracy Top5: 0.8725
pruning conv8 feature map 141 ...
Loss:  1.9106843388080597 Accuracy Top1: 0.52

Loss:  2.0431022000312806 Accuracy Top1: 0.495 Accuracy Top5: 0.866875
pruning conv8 feature map 211 ...
Loss:  2.002089240550995 Accuracy Top1: 0.49875 Accuracy Top5: 0.864375
pruning conv8 feature map 212 ...
Loss:  2.02299729347229 Accuracy Top1: 0.498125 Accuracy Top5: 0.86375
pruning conv8 feature map 213 ...
Loss:  2.03475177526474 Accuracy Top1: 0.48875 Accuracy Top5: 0.865
pruning conv8 feature map 214 ...
Loss:  1.8932553148269653 Accuracy Top1: 0.515625 Accuracy Top5: 0.876875
pruning conv8 feature map 215 ...
Loss:  2.0921590924263 Accuracy Top1: 0.4875 Accuracy Top5: 0.870625
pruning conv8 feature map 216 ...
Loss:  1.929418567419052 Accuracy Top1: 0.518125 Accuracy Top5: 0.8775
pruning conv8 feature map 217 ...
Loss:  1.950508623123169 Accuracy Top1: 0.515 Accuracy Top5: 0.87125
pruning conv8 feature map 218 ...
Loss:  2.04551922082901 Accuracy Top1: 0.496875 Accuracy Top5: 0.863125
pruning conv8 feature map 219 ...
Loss:  2.0786232590675353 Accuracy Top1: 0.486875 Accurac

In [7]:
test_pruned_fm(prototxt, caffemodel, 'conv9', 100, 0.25)
test_pruned_fm(prototxt, caffemodel, 'conv10',100, 0.25)
test_pruned_fm(prototxt, caffemodel, 'conv11',100, 0.25)


pruning conv9 feature map 0 ...
Loss:  2.03054288983345 Accuracy Top1: 0.496875 Accuracy Top5: 0.865
pruning conv9 feature map 1 ...
Loss:  2.0378096497058866 Accuracy Top1: 0.494375 Accuracy Top5: 0.865
pruning conv9 feature map 2 ...
Loss:  1.9994318783283234 Accuracy Top1: 0.4990625 Accuracy Top5: 0.87375
pruning conv9 feature map 3 ...
Loss:  1.9254510474205018 Accuracy Top1: 0.5175 Accuracy Top5: 0.8759375
pruning conv9 feature map 4 ...
Loss:  2.101870641708374 Accuracy Top1: 0.4890625 Accuracy Top5: 0.858125
pruning conv9 feature map 5 ...
Loss:  2.0093003261089324 Accuracy Top1: 0.4965625 Accuracy Top5: 0.8653125
pruning conv9 feature map 6 ...
Loss:  1.983815859556198 Accuracy Top1: 0.5021875 Accuracy Top5: 0.875
pruning conv9 feature map 7 ...
Loss:  1.9771462631225587 Accuracy Top1: 0.506875 Accuracy Top5: 0.8734375
pruning conv9 feature map 8 ...
Loss:  2.0441921150684355 Accuracy Top1: 0.495 Accuracy Top5: 0.8609375
pruning conv9 feature map 9 ...
Loss:  1.9627591758966445

Loss:  2.000610884428024 Accuracy Top1: 0.49875 Accuracy Top5: 0.87375
pruning conv9 feature map 78 ...
Loss:  1.9147280716896058 Accuracy Top1: 0.516875 Accuracy Top5: 0.875625
pruning conv9 feature map 79 ...
Loss:  2.075155463218689 Accuracy Top1: 0.49375 Accuracy Top5: 0.8603125
pruning conv9 feature map 80 ...
Loss:  2.0191638231277467 Accuracy Top1: 0.4934375 Accuracy Top5: 0.863125
pruning conv9 feature map 81 ...
Loss:  1.987175475358963 Accuracy Top1: 0.5046875 Accuracy Top5: 0.875
pruning conv9 feature map 82 ...
Loss:  1.9709613090753555 Accuracy Top1: 0.50875 Accuracy Top5: 0.8740625
pruning conv9 feature map 83 ...
Loss:  2.0515505802631377 Accuracy Top1: 0.4928125 Accuracy Top5: 0.86125
pruning conv9 feature map 84 ...
Loss:  1.9698268592357635 Accuracy Top1: 0.4975 Accuracy Top5: 0.870625
pruning conv9 feature map 85 ...
Loss:  2.0196431320905686 Accuracy Top1: 0.5009375 Accuracy Top5: 0.8721875
pruning conv9 feature map 86 ...
Loss:  1.9970993387699127 Accuracy Top1: 0.

Loss:  2.0334014356136323 Accuracy Top1: 0.4984375 Accuracy Top5: 0.8640625
pruning conv9 feature map 155 ...
Loss:  2.0270046055316926 Accuracy Top1: 0.4934375 Accuracy Top5: 0.8615625
pruning conv9 feature map 156 ...
Loss:  1.9828372752666474 Accuracy Top1: 0.506875 Accuracy Top5: 0.8753125
pruning conv9 feature map 157 ...
Loss:  1.9760018658638 Accuracy Top1: 0.50875 Accuracy Top5: 0.8728125
pruning conv9 feature map 158 ...
Loss:  2.057260284423828 Accuracy Top1: 0.4928125 Accuracy Top5: 0.86125
pruning conv9 feature map 159 ...
Loss:  1.9596877312660217 Accuracy Top1: 0.5028125 Accuracy Top5: 0.8715625
pruning conv9 feature map 160 ...
Loss:  2.0195812249183653 Accuracy Top1: 0.501875 Accuracy Top5: 0.875
pruning conv9 feature map 161 ...
Loss:  2.0142510974407197 Accuracy Top1: 0.503125 Accuracy Top5: 0.8634375
pruning conv9 feature map 162 ...
Loss:  2.0462430250644683 Accuracy Top1: 0.4909375 Accuracy Top5: 0.85875
pruning conv9 feature map 163 ...
Loss:  1.9805296677350999 A

Loss:  2.0025677067041396 Accuracy Top1: 0.494375 Accuracy Top5: 0.8628125
pruning conv9 feature map 231 ...
Loss:  1.9861770534515382 Accuracy Top1: 0.505 Accuracy Top5: 0.875
pruning conv9 feature map 232 ...
Loss:  1.9733431112766266 Accuracy Top1: 0.5075 Accuracy Top5: 0.8725
pruning conv9 feature map 233 ...
Loss:  2.0350122332572935 Accuracy Top1: 0.498125 Accuracy Top5: 0.8628125
pruning conv9 feature map 234 ...
Loss:  1.9541906106472016 Accuracy Top1: 0.50125 Accuracy Top5: 0.8709375
pruning conv9 feature map 235 ...
Loss:  2.019559389948845 Accuracy Top1: 0.5021875 Accuracy Top5: 0.8740625
pruning conv9 feature map 236 ...
Loss:  2.0214881253242494 Accuracy Top1: 0.50375 Accuracy Top5: 0.86375
pruning conv9 feature map 237 ...
Loss:  2.0355485785007477 Accuracy Top1: 0.493125 Accuracy Top5: 0.8609375
pruning conv9 feature map 238 ...
Loss:  1.9790714114904404 Accuracy Top1: 0.4990625 Accuracy Top5: 0.8728125
pruning conv9 feature map 239 ...
Loss:  1.969375081062317 Accuracy 

Loss:  1.9707620334625244 Accuracy Top1: 0.508125 Accuracy Top5: 0.873125
pruning conv9 feature map 308 ...
Loss:  2.0548047256469726 Accuracy Top1: 0.49375 Accuracy Top5: 0.861875
pruning conv9 feature map 309 ...
Loss:  1.9660981261730195 Accuracy Top1: 0.500625 Accuracy Top5: 0.8709375
pruning conv9 feature map 310 ...
Loss:  2.006164846420288 Accuracy Top1: 0.5034375 Accuracy Top5: 0.8753125
pruning conv9 feature map 311 ...
Loss:  2.0018394947052003 Accuracy Top1: 0.50375 Accuracy Top5: 0.8653125
pruning conv9 feature map 312 ...
Loss:  2.0423178493976595 Accuracy Top1: 0.493125 Accuracy Top5: 0.860625
pruning conv9 feature map 313 ...
Loss:  1.9823497027158736 Accuracy Top1: 0.49625 Accuracy Top5: 0.8728125
pruning conv9 feature map 314 ...
Loss:  1.963936539888382 Accuracy Top1: 0.514375 Accuracy Top5: 0.8753125
pruning conv9 feature map 315 ...
Loss:  2.042017287015915 Accuracy Top1: 0.49625 Accuracy Top5: 0.86125
pruning conv9 feature map 316 ...
Loss:  2.028253008723259 Accur

...

Loss:  1.9857095396518707 Accuracy Top1: 0.5034375 Accuracy Top5: 0.87375
pruning conv11 feature map 187 ...
Loss:  1.9845289659500123 Accuracy Top1: 0.5053125 Accuracy Top5: 0.873125
pruning conv11 feature map 188 ...
Loss:  2.0501373970508574 Accuracy Top1: 0.494375 Accuracy Top5: 0.86125
pruning conv11 feature map 189 ...
Loss:  1.9632093286514283 Accuracy Top1: 0.4990625 Accuracy Top5: 0.870625
pruning conv11 feature map 190 ...
Loss:  2.019169083237648 Accuracy Top1: 0.5028125 Accuracy Top5: 0.8746875
pruning conv11 feature map 191 ...
Loss:  2.00264688372612 Accuracy Top1: 0.505625 Accuracy Top5: 0.866875
pruning conv11 feature map 192 ...
Loss:  2.0550743782520295 Accuracy Top1: 0.4915625 Accuracy Top5: 0.8575
pruning conv11 feature map 193 ...
Loss:  1.9762144362926484 Accuracy Top1: 0.4984375 Accuracy Top5: 0.87375
pruning conv11 feature map 194 ...
Loss:  1.9700074005126953 Accuracy Top1: 0.5134375 Accuracy Top5: 0.87375
pruning conv11 feature map 195 ...
Loss:  2.04582686543

Loss:  1.9746319651603699 Accuracy Top1: 0.5078125 Accuracy Top5: 0.873125
pruning conv11 feature map 263 ...
Loss:  2.042428379058838 Accuracy Top1: 0.496875 Accuracy Top5: 0.8609375
pruning conv11 feature map 264 ...
Loss:  1.9778353047370911 Accuracy Top1: 0.4978125 Accuracy Top5: 0.8703125
pruning conv11 feature map 265 ...
Loss:  1.9577060890197755 Accuracy Top1: 0.509375 Accuracy Top5: 0.8778125
pruning conv11 feature map 266 ...
Loss:  1.9945030629634857 Accuracy Top1: 0.5071875 Accuracy Top5: 0.8678125
pruning conv11 feature map 267 ...
Loss:  2.0481256151199343 Accuracy Top1: 0.4934375 Accuracy Top5: 0.8603125
pruning conv11 feature map 268 ...
Loss:  1.9746369063854217 Accuracy Top1: 0.4996875 Accuracy Top5: 0.8734375
pruning conv11 feature map 269 ...
Loss:  1.9669055461883544 Accuracy Top1: 0.51375 Accuracy Top5: 0.8728125
pruning conv11 feature map 270 ...
Loss:  2.0378517150878905 Accuracy Top1: 0.49625 Accuracy Top5: 0.864375
pruning conv11 feature map 271 ...
Loss:  2.0

Loss:  2.0449128687381743 Accuracy Top1: 0.495 Accuracy Top5: 0.8615625
pruning conv11 feature map 339 ...
Loss:  1.9709567546844482 Accuracy Top1: 0.499375 Accuracy Top5: 0.8703125
pruning conv11 feature map 340 ...
Loss:  2.014038362503052 Accuracy Top1: 0.501875 Accuracy Top5: 0.8740625
pruning conv11 feature map 341 ...
Loss:  1.9987204432487489 Accuracy Top1: 0.505625 Accuracy Top5: 0.86625
pruning conv11 feature map 342 ...
Loss:  2.040640392303467 Accuracy Top1: 0.4946875 Accuracy Top5: 0.8615625
pruning conv11 feature map 343 ...
Loss:  1.9799778002500534 Accuracy Top1: 0.498125 Accuracy Top5: 0.8734375
pruning conv11 feature map 344 ...
Loss:  1.969935028553009 Accuracy Top1: 0.513125 Accuracy Top5: 0.875
pruning conv11 feature map 345 ...
Loss:  2.039955153465271 Accuracy Top1: 0.495 Accuracy Top5: 0.8640625
pruning conv11 feature map 346 ...
Loss:  2.035663821101189 Accuracy Top1: 0.4928125 Accuracy Top5: 0.865625
pruning conv11 feature map 347 ...
Loss:  2.000427166223526 A

Loss:  1.963115199804306 Accuracy Top1: 0.5015625 Accuracy Top5: 0.871875
pruning conv11 feature map 415 ...
Loss:  2.013582934141159 Accuracy Top1: 0.5021875 Accuracy Top5: 0.8753125
pruning conv11 feature map 416 ...
Loss:  2.0006405103206633 Accuracy Top1: 0.505 Accuracy Top5: 0.866875
pruning conv11 feature map 417 ...
Loss:  2.0470334732532502 Accuracy Top1: 0.4928125 Accuracy Top5: 0.8615625
pruning conv11 feature map 418 ...
Loss:  1.9813819587230683 Accuracy Top1: 0.4975 Accuracy Top5: 0.8740625
pruning conv11 feature map 419 ...
Loss:  1.9596416610479355 Accuracy Top1: 0.5128125 Accuracy Top5: 0.874375
pruning conv11 feature map 420 ...
Loss:  2.0417621290683745 Accuracy Top1: 0.4946875 Accuracy Top5: 0.861875
pruning conv11 feature map 421 ...
Loss:  2.0372279512882234 Accuracy Top1: 0.493125 Accuracy Top5: 0.865625
pruning conv11 feature map 422 ...
Loss:  1.992983512878418 Accuracy Top1: 0.4996875 Accuracy Top5: 0.874375
pruning conv11 feature map 423 ...
Loss:  1.931934916

Loss:  2.0086462354660033 Accuracy Top1: 0.5034375 Accuracy Top5: 0.873125
pruning conv11 feature map 491 ...
Loss:  1.9998721086978912 Accuracy Top1: 0.506875 Accuracy Top5: 0.8659375
pruning conv11 feature map 492 ...
Loss:  2.0211929523944856 Accuracy Top1: 0.496875 Accuracy Top5: 0.86375
pruning conv11 feature map 493 ...
Loss:  1.9845854783058166 Accuracy Top1: 0.4971875 Accuracy Top5: 0.87375
pruning conv11 feature map 494 ...
Loss:  1.9651458436250686 Accuracy Top1: 0.5128125 Accuracy Top5: 0.8740625
pruning conv11 feature map 495 ...
Loss:  2.0437459337711332 Accuracy Top1: 0.494375 Accuracy Top5: 0.863125
pruning conv11 feature map 496 ...
Loss:  2.0413638174533846 Accuracy Top1: 0.49125 Accuracy Top5: 0.864375
pruning conv11 feature map 497 ...
Loss:  2.00325492143631 Accuracy Top1: 0.499375 Accuracy Top5: 0.874375
pruning conv11 feature map 498 ...
Loss:  1.9432176804542542 Accuracy Top1: 0.5159375 Accuracy Top5: 0.8734375
pruning conv11 feature map 499 ...
Loss:  2.05683619

In [9]:
log=open("Darkent.log",'a+') 
test_pruned_fm(prototxt, caffemodel, 'conv9', 100, 0.375)
test_pruned_fm(prototxt, caffemodel, 'conv10',100, 0.375)
test_pruned_fm(prototxt, caffemodel, 'conv11',100, 0.375)
log.close()
#看一下之前不对是因为样本数量小还是砍的太多

pruning conv9 feature map 0 ...
Loss:  2.03054288983345 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.865
pruning conv9 feature map 1 ...
Loss:  2.0378096497058866 , Accuracy Top1: 0.494375 , Accuracy Top5: 0.865
pruning conv9 feature map 2 ...
Loss:  1.9994318783283234 , Accuracy Top1: 0.4990625 , Accuracy Top5: 0.87375
pruning conv9 feature map 3 ...
Loss:  1.9254510474205018 , Accuracy Top1: 0.5175 , Accuracy Top5: 0.8759375
pruning conv9 feature map 4 ...
Loss:  2.101870641708374 , Accuracy Top1: 0.4890625 , Accuracy Top5: 0.858125
pruning conv9 feature map 5 ...
Loss:  2.0093003261089324 , Accuracy Top1: 0.4965625 , Accuracy Top5: 0.8653125
pruning conv9 feature map 6 ...
Loss:  1.983815859556198 , Accuracy Top1: 0.5021875 , Accuracy Top5: 0.875
pruning conv9 feature map 7 ...
Loss:  1.9771462631225587 , Accuracy Top1: 0.506875 , Accuracy Top5: 0.8734375
pruning conv9 feature map 8 ...
Loss:  2.0441921150684355 , Accuracy Top1: 0.495 , Accuracy Top5: 0.8609375
pruning conv9 feature

Loss:  1.9447422105073928 , Accuracy Top1: 0.5153125 , Accuracy Top5: 0.875625
pruning conv9 feature map 75 ...
Loss:  2.040986157655716 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.8640625
pruning conv9 feature map 76 ...
Loss:  2.048906954526901 , Accuracy Top1: 0.489375 , Accuracy Top5: 0.861875
pruning conv9 feature map 77 ...
Loss:  2.000610884428024 , Accuracy Top1: 0.49875 , Accuracy Top5: 0.87375
pruning conv9 feature map 78 ...
Loss:  1.9147280716896058 , Accuracy Top1: 0.516875 , Accuracy Top5: 0.875625
pruning conv9 feature map 79 ...
Loss:  2.075155463218689 , Accuracy Top1: 0.49375 , Accuracy Top5: 0.8603125
pruning conv9 feature map 80 ...
Loss:  2.0191638231277467 , Accuracy Top1: 0.4934375 , Accuracy Top5: 0.863125
pruning conv9 feature map 81 ...
Loss:  1.987175475358963 , Accuracy Top1: 0.5046875 , Accuracy Top5: 0.875
pruning conv9 feature map 82 ...
Loss:  1.9709613090753555 , Accuracy Top1: 0.50875 , Accuracy Top5: 0.8740625
pruning conv9 feature map 83 ...
Loss:  

Loss:  1.972782308459282 , Accuracy Top1: 0.5009375 , Accuracy Top5: 0.87375
pruning conv9 feature map 149 ...
Loss:  1.9651760280132293 , Accuracy Top1: 0.5146875 , Accuracy Top5: 0.8740625
pruning conv9 feature map 150 ...
Loss:  2.0532699275016784 , Accuracy Top1: 0.49375 , Accuracy Top5: 0.8621875
pruning conv9 feature map 151 ...
Loss:  2.0414906871318816 , Accuracy Top1: 0.491875 , Accuracy Top5: 0.8640625
pruning conv9 feature map 152 ...
Loss:  2.0012697422504426 , Accuracy Top1: 0.49875 , Accuracy Top5: 0.8728125
pruning conv9 feature map 153 ...
Loss:  1.9386191427707673 , Accuracy Top1: 0.5171875 , Accuracy Top5: 0.874375
pruning conv9 feature map 154 ...
Loss:  2.0334014356136323 , Accuracy Top1: 0.4984375 , Accuracy Top5: 0.8640625
pruning conv9 feature map 155 ...
Loss:  2.0270046055316926 , Accuracy Top1: 0.4934375 , Accuracy Top5: 0.8615625
pruning conv9 feature map 156 ...
Loss:  1.9828372752666474 , Accuracy Top1: 0.506875 , Accuracy Top5: 0.8753125
pruning conv9 feat

Loss:  2.0448723471164705 , Accuracy Top1: 0.4934375 , Accuracy Top5: 0.8575
pruning conv9 feature map 223 ...
Loss:  1.980211157798767 , Accuracy Top1: 0.4978125 , Accuracy Top5: 0.8734375
pruning conv9 feature map 224 ...
Loss:  1.9907470953464508 , Accuracy Top1: 0.5084375 , Accuracy Top5: 0.8703125
pruning conv9 feature map 225 ...
Loss:  2.055800518989563 , Accuracy Top1: 0.4953125 , Accuracy Top5: 0.863125
pruning conv9 feature map 226 ...
Loss:  2.035656063556671 , Accuracy Top1: 0.493125 , Accuracy Top5: 0.865
pruning conv9 feature map 227 ...
Loss:  2.0095426511764525 , Accuracy Top1: 0.4978125 , Accuracy Top5: 0.8740625
pruning conv9 feature map 228 ...
Loss:  1.920176540017128 , Accuracy Top1: 0.51875 , Accuracy Top5: 0.8771875
pruning conv9 feature map 229 ...
Loss:  2.0514669287204743 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.8603125
pruning conv9 feature map 230 ...
Loss:  2.0025677067041396 , Accuracy Top1: 0.494375 , Accuracy Top5: 0.8628125
pruning conv9 feature map

Loss:  2.002116496562958 , Accuracy Top1: 0.5046875 , Accuracy Top5: 0.8653125
pruning conv9 feature map 297 ...
Loss:  2.055745491981506 , Accuracy Top1: 0.4909375 , Accuracy Top5: 0.8584375
pruning conv9 feature map 298 ...
Loss:  1.9829789847135544 , Accuracy Top1: 0.4984375 , Accuracy Top5: 0.87375
pruning conv9 feature map 299 ...
Loss:  1.9499618846178055 , Accuracy Top1: 0.51625 , Accuracy Top5: 0.87625
pruning conv9 feature map 300 ...
Loss:  2.0421556627750395 , Accuracy Top1: 0.4971875 , Accuracy Top5: 0.8634375
pruning conv9 feature map 301 ...
Loss:  2.042531419992447 , Accuracy Top1: 0.4921875 , Accuracy Top5: 0.8646875
pruning conv9 feature map 302 ...
Loss:  1.9980267870426178 , Accuracy Top1: 0.4996875 , Accuracy Top5: 0.8715625
pruning conv9 feature map 303 ...
Loss:  1.9472197878360749 , Accuracy Top1: 0.5159375 , Accuracy Top5: 0.8740625
pruning conv9 feature map 304 ...
Loss:  2.0561724865436553 , Accuracy Top1: 0.495625 , Accuracy Top5: 0.8609375
pruning conv9 feat

...

Loss:  2.0526948761940003 , Accuracy Top1: 0.4940625 , Accuracy Top5: 0.860625
pruning conv11 feature map 215 ...
Loss:  2.0167405569553374 , Accuracy Top1: 0.4946875 , Accuracy Top5: 0.8640625
pruning conv11 feature map 216 ...
Loss:  1.975764937400818 , Accuracy Top1: 0.5090625 , Accuracy Top5: 0.8765625
pruning conv11 feature map 217 ...
Loss:  1.9759219819307328 , Accuracy Top1: 0.5078125 , Accuracy Top5: 0.8728125
pruning conv11 feature map 218 ...
Loss:  2.0557786667346956 , Accuracy Top1: 0.4934375 , Accuracy Top5: 0.861875
pruning conv11 feature map 219 ...
Loss:  1.960932377576828 , Accuracy Top1: 0.50375 , Accuracy Top5: 0.87125
pruning conv11 feature map 220 ...
Loss:  2.014903371334076 , Accuracy Top1: 0.503125 , Accuracy Top5: 0.8725
pruning conv11 feature map 221 ...
Loss:  1.9949869298934937 , Accuracy Top1: 0.5071875 , Accuracy Top5: 0.8675
pruning conv11 feature map 222 ...
Loss:  2.047521721124649 , Accuracy Top1: 0.4928125 , Accuracy Top5: 0.860625
pruning conv11 fea

Loss:  1.9838837015628814 , Accuracy Top1: 0.5040625 , Accuracy Top5: 0.875
pruning conv11 feature map 288 ...
Loss:  1.9461445128917694 , Accuracy Top1: 0.516875 , Accuracy Top5: 0.8740625
pruning conv11 feature map 289 ...
Loss:  2.05987140417099 , Accuracy Top1: 0.4940625 , Accuracy Top5: 0.860625
pruning conv11 feature map 290 ...
Loss:  2.005232934355736 , Accuracy Top1: 0.494375 , Accuracy Top5: 0.864375
pruning conv11 feature map 291 ...
Loss:  1.9802742755413056 , Accuracy Top1: 0.5065625 , Accuracy Top5: 0.8728125
pruning conv11 feature map 292 ...
Loss:  1.976351781487465 , Accuracy Top1: 0.5078125 , Accuracy Top5: 0.8728125
pruning conv11 feature map 293 ...
Loss:  2.0470556557178496 , Accuracy Top1: 0.4953125 , Accuracy Top5: 0.8609375
pruning conv11 feature map 294 ...
Loss:  1.9835374975204467 , Accuracy Top1: 0.4971875 , Accuracy Top5: 0.86875
pruning conv11 feature map 295 ...
Loss:  2.0291856569051743 , Accuracy Top1: 0.499375 , Accuracy Top5: 0.8721875
pruning conv11 

Loss:  2.048126244544983 , Accuracy Top1: 0.495 , Accuracy Top5: 0.861875
pruning conv11 feature map 361 ...
Loss:  2.0390898126363752 , Accuracy Top1: 0.4921875 , Accuracy Top5: 0.865
pruning conv11 feature map 362 ...
Loss:  2.015889208316803 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.8696875
pruning conv11 feature map 363 ...
Loss:  1.9389834427833557 , Accuracy Top1: 0.5171875 , Accuracy Top5: 0.875
pruning conv11 feature map 364 ...
Loss:  2.0550433337688445 , Accuracy Top1: 0.4940625 , Accuracy Top5: 0.8603125
pruning conv11 feature map 365 ...
Loss:  2.0161300909519198 , Accuracy Top1: 0.4946875 , Accuracy Top5: 0.8634375
pruning conv11 feature map 366 ...
Loss:  1.9863675844669342 , Accuracy Top1: 0.50625 , Accuracy Top5: 0.875
pruning conv11 feature map 367 ...
Loss:  1.9757769155502318 , Accuracy Top1: 0.5084375 , Accuracy Top5: 0.871875
pruning conv11 feature map 368 ...
Loss:  2.0371213614940644 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.860625
pruning conv11 feature ma

Loss:  1.9672450876235963 , Accuracy Top1: 0.5003125 , Accuracy Top5: 0.8734375
pruning conv11 feature map 434 ...
Loss:  1.968290069103241 , Accuracy Top1: 0.5128125 , Accuracy Top5: 0.8753125
pruning conv11 feature map 435 ...
Loss:  2.0476040267944335 , Accuracy Top1: 0.4946875 , Accuracy Top5: 0.8621875
pruning conv11 feature map 436 ...
Loss:  2.0399273765087127 , Accuracy Top1: 0.491875 , Accuracy Top5: 0.8646875
pruning conv11 feature map 437 ...
Loss:  2.0014647459983825 , Accuracy Top1: 0.499375 , Accuracy Top5: 0.8740625
pruning conv11 feature map 438 ...
Loss:  1.9460147154331207 , Accuracy Top1: 0.5171875 , Accuracy Top5: 0.8725
pruning conv11 feature map 439 ...
Loss:  2.058037438392639 , Accuracy Top1: 0.494375 , Accuracy Top5: 0.8603125
pruning conv11 feature map 440 ...
Loss:  2.0262523704767226 , Accuracy Top1: 0.4925 , Accuracy Top5: 0.865
pruning conv11 feature map 441 ...
Loss:  1.9742039787769317 , Accuracy Top1: 0.5071875 , Accuracy Top5: 0.875625
pruning conv11 f

Loss:  1.9983410227298737 , Accuracy Top1: 0.504375 , Accuracy Top5: 0.8671875
pruning conv11 feature map 507 ...
Loss:  2.0435941076278685 , Accuracy Top1: 0.4934375 , Accuracy Top5: 0.86
pruning conv11 feature map 508 ...
Loss:  1.9768039298057556 , Accuracy Top1: 0.49875 , Accuracy Top5: 0.873125
pruning conv11 feature map 509 ...
Loss:  1.9635783004760743 , Accuracy Top1: 0.513125 , Accuracy Top5: 0.8740625
pruning conv11 feature map 510 ...
Loss:  2.06386301279068 , Accuracy Top1: 0.4925 , Accuracy Top5: 0.8628125
pruning conv11 feature map 511 ...
Loss:  2.008770365715027 , Accuracy Top1: 0.4965625 , Accuracy Top5: 0.8671875
cut unimportant features...
prune basing on top1...
fm to be pruned:  [  3 228  93 108 468  78 393 318 423 483 213 198  63  33 438 363 453  18
 138 408 153 288 254 123 378 243 303 273 498 329 348 183 333  48 258 479
 168 464 224 269  14 179 194  89 509  74 149 235 209 344  29 419 434 374
 494  59 314 449 389 404  44 134 164 104 359 239 284 119 299 157 131 142

In [10]:
log=open("Darkent.log",'a+') 
test_pruned_fm(prototxt, caffemodel, 'conv12', 100, 0.375)
test_pruned_fm(prototxt, caffemodel, 'conv13', 100, 0.375)
test_pruned_fm(prototxt, caffemodel, 'conv14', 100, 0.375)
log.close()

pruning conv12 feature map 0 ...
Loss:  2.0647344732284547 , Accuracy Top1: 0.4884375 , Accuracy Top5: 0.8590625
pruning conv12 feature map 1 ...
Loss:  2.0529740381240846 , Accuracy Top1: 0.490625 , Accuracy Top5: 0.8634375
pruning conv12 feature map 2 ...
Loss:  1.9975670421123504 , Accuracy Top1: 0.4978125 , Accuracy Top5: 0.8728125
pruning conv12 feature map 3 ...
Loss:  1.9386848241090775 , Accuracy Top1: 0.5165625 , Accuracy Top5: 0.8740625
pruning conv12 feature map 4 ...
Loss:  2.0544585275650022 , Accuracy Top1: 0.4940625 , Accuracy Top5: 0.8596875
pruning conv12 feature map 5 ...
Loss:  1.9930219674110412 , Accuracy Top1: 0.4990625 , Accuracy Top5: 0.8634375
pruning conv12 feature map 6 ...
Loss:  1.9843320953845978 , Accuracy Top1: 0.5084375 , Accuracy Top5: 0.871875
pruning conv12 feature map 7 ...
Loss:  1.963368079662323 , Accuracy Top1: 0.5084375 , Accuracy Top5: 0.8740625
pruning conv12 feature map 8 ...
Loss:  2.0435030019283293 , Accuracy Top1: 0.495 , Accuracy Top5: 

Loss:  1.9021554470062256 , Accuracy Top1: 0.524375 , Accuracy Top5: 0.8771875
pruning conv12 feature map 75 ...
Loss:  2.050985881090164 , Accuracy Top1: 0.4921875 , Accuracy Top5: 0.8625
pruning conv12 feature map 76 ...
Loss:  2.03035371363163 , Accuracy Top1: 0.49375 , Accuracy Top5: 0.86625
pruning conv12 feature map 77 ...
Loss:  1.9950461685657501 , Accuracy Top1: 0.4990625 , Accuracy Top5: 0.874375
pruning conv12 feature map 78 ...
Loss:  1.9291725003719329 , Accuracy Top1: 0.5184375 , Accuracy Top5: 0.875
pruning conv12 feature map 79 ...
Loss:  2.0504614329338073 , Accuracy Top1: 0.493125 , Accuracy Top5: 0.86
pruning conv12 feature map 80 ...
Loss:  2.0155618357658387 , Accuracy Top1: 0.493125 , Accuracy Top5: 0.8634375
pruning conv12 feature map 81 ...
Loss:  1.9858256685733795 , Accuracy Top1: 0.5046875 , Accuracy Top5: 0.8740625
pruning conv12 feature map 82 ...
Loss:  1.9497948336601256 , Accuracy Top1: 0.50625 , Accuracy Top5: 0.8734375
pruning conv12 feature map 83 ...

Loss:  1.9808923095464706 , Accuracy Top1: 0.4959375 , Accuracy Top5: 0.871875
pruning conv12 feature map 149 ...
Loss:  1.947381972670555 , Accuracy Top1: 0.514375 , Accuracy Top5: 0.875625
pruning conv12 feature map 150 ...
Loss:  2.0454627799987795 , Accuracy Top1: 0.4946875 , Accuracy Top5: 0.8634375
pruning conv12 feature map 151 ...
Loss:  2.0456411457061767 , Accuracy Top1: 0.4934375 , Accuracy Top5: 0.8646875
pruning conv12 feature map 152 ...
Loss:  2.0097783648967744 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.8715625
pruning conv12 feature map 153 ...
Loss:  1.9447125935554503 , Accuracy Top1: 0.515 , Accuracy Top5: 0.875
pruning conv12 feature map 154 ...
Loss:  2.055880550146103 , Accuracy Top1: 0.4940625 , Accuracy Top5: 0.861875
pruning conv12 feature map 155 ...
Loss:  1.9997661513090135 , Accuracy Top1: 0.4959375 , Accuracy Top5: 0.86625
pruning conv12 feature map 156 ...
Loss:  1.9978348922729492 , Accuracy Top1: 0.50375 , Accuracy Top5: 0.8725
pruning conv12 feature m

Loss:  1.9650051522254943 , Accuracy Top1: 0.5075 , Accuracy Top5: 0.87
pruning conv12 feature map 222 ...
Loss:  2.0433149266242983 , Accuracy Top1: 0.493125 , Accuracy Top5: 0.86125
pruning conv12 feature map 223 ...
Loss:  1.9805684620141983 , Accuracy Top1: 0.4978125 , Accuracy Top5: 0.8740625
pruning conv12 feature map 224 ...
Loss:  1.951754967570305 , Accuracy Top1: 0.514375 , Accuracy Top5: 0.87625
pruning conv12 feature map 225 ...
Loss:  2.044733123779297 , Accuracy Top1: 0.494375 , Accuracy Top5: 0.86125
pruning conv12 feature map 226 ...
Loss:  2.0439186626672745 , Accuracy Top1: 0.488125 , Accuracy Top5: 0.8640625
pruning conv12 feature map 227 ...
Loss:  2.0024923276901245 , Accuracy Top1: 0.499375 , Accuracy Top5: 0.873125
pruning conv12 feature map 228 ...
Loss:  1.9425245928764343 , Accuracy Top1: 0.5175 , Accuracy Top5: 0.8740625
pruning conv12 feature map 229 ...
Loss:  2.009060015678406 , Accuracy Top1: 0.501875 , Accuracy Top5: 0.866875
pruning conv12 feature map 2

Loss:  2.06344389796257 , Accuracy Top1: 0.490625 , Accuracy Top5: 0.8603125
pruning conv13 feature map 20 ...
Loss:  2.0253094440698622 , Accuracy Top1: 0.494375 , Accuracy Top5: 0.86375
pruning conv13 feature map 21 ...
Loss:  1.9920080232620239 , Accuracy Top1: 0.50375 , Accuracy Top5: 0.874375
pruning conv13 feature map 22 ...
Loss:  1.9802080273628235 , Accuracy Top1: 0.50625 , Accuracy Top5: 0.873125
pruning conv13 feature map 23 ...
Loss:  2.039065833091736 , Accuracy Top1: 0.4953125 , Accuracy Top5: 0.8634375
pruning conv13 feature map 24 ...
Loss:  1.9654022079706193 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.8703125
pruning conv13 feature map 25 ...
Loss:  2.017711944580078 , Accuracy Top1: 0.5028125 , Accuracy Top5: 0.874375
pruning conv13 feature map 26 ...
Loss:  1.9991850113868714 , Accuracy Top1: 0.505625 , Accuracy Top5: 0.86625
pruning conv13 feature map 27 ...
Loss:  2.0368490481376647 , Accuracy Top1: 0.494375 , Accuracy Top5: 0.8615625
pruning conv13 feature map 2

...

Loss:  2.05565007686615 , Accuracy Top1: 0.4934375 , Accuracy Top5: 0.8640625
pruning conv14 feature map 736 ...
Loss:  2.0268867218494417 , Accuracy Top1: 0.493125 , Accuracy Top5: 0.8665625
pruning conv14 feature map 737 ...
Loss:  1.9921410357952118 , Accuracy Top1: 0.4996875 , Accuracy Top5: 0.8753125
pruning conv14 feature map 738 ...
Loss:  1.929863491654396 , Accuracy Top1: 0.5190625 , Accuracy Top5: 0.8753125
pruning conv14 feature map 739 ...
Loss:  2.052745599746704 , Accuracy Top1: 0.4934375 , Accuracy Top5: 0.8628125
pruning conv14 feature map 740 ...
Loss:  2.0194121491909027 , Accuracy Top1: 0.4946875 , Accuracy Top5: 0.8634375
pruning conv14 feature map 741 ...
Loss:  1.983324143886566 , Accuracy Top1: 0.505625 , Accuracy Top5: 0.874375
pruning conv14 feature map 742 ...
Loss:  1.9748792791366576 , Accuracy Top1: 0.5084375 , Accuracy Top5: 0.8734375
pruning conv14 feature map 743 ...
Loss:  2.0521822226047517 , Accuracy Top1: 0.4934375 , Accuracy Top5: 0.86125
pruning co

Loss:  1.9842575812339782 , Accuracy Top1: 0.495625 , Accuracy Top5: 0.8725
pruning conv14 feature map 809 ...
Loss:  1.9684318178892135 , Accuracy Top1: 0.5128125 , Accuracy Top5: 0.8740625
pruning conv14 feature map 810 ...
Loss:  2.0396316862106323 , Accuracy Top1: 0.495625 , Accuracy Top5: 0.863125
pruning conv14 feature map 811 ...
Loss:  2.0404024535417555 , Accuracy Top1: 0.4921875 , Accuracy Top5: 0.8640625
pruning conv14 feature map 812 ...
Loss:  2.0031556808948516 , Accuracy Top1: 0.5 , Accuracy Top5: 0.8746875
pruning conv14 feature map 813 ...
Loss:  1.9429332029819488 , Accuracy Top1: 0.51625 , Accuracy Top5: 0.874375
pruning conv14 feature map 814 ...
Loss:  2.053042868375778 , Accuracy Top1: 0.4946875 , Accuracy Top5: 0.8628125
pruning conv14 feature map 815 ...
Loss:  2.023550671339035 , Accuracy Top1: 0.4953125 , Accuracy Top5: 0.8625
pruning conv14 feature map 816 ...
Loss:  1.9895628988742828 , Accuracy Top1: 0.504375 , Accuracy Top5: 0.8728125
pruning conv14 featur

Loss:  2.0010352551937105 , Accuracy Top1: 0.505 , Accuracy Top5: 0.86625
pruning conv14 feature map 882 ...
Loss:  2.0321441197395327 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.8615625
pruning conv14 feature map 883 ...
Loss:  1.9821928137540816 , Accuracy Top1: 0.49875 , Accuracy Top5: 0.8734375
pruning conv14 feature map 884 ...
Loss:  1.9767401307821273 , Accuracy Top1: 0.5109375 , Accuracy Top5: 0.871875
pruning conv14 feature map 885 ...
Loss:  2.048237291574478 , Accuracy Top1: 0.4946875 , Accuracy Top5: 0.8625
pruning conv14 feature map 886 ...
Loss:  2.037579932808876 , Accuracy Top1: 0.491875 , Accuracy Top5: 0.86625
pruning conv14 feature map 887 ...
Loss:  2.0075712049007417 , Accuracy Top1: 0.4965625 , Accuracy Top5: 0.8728125
pruning conv14 feature map 888 ...
Loss:  1.9415122091770172 , Accuracy Top1: 0.5178125 , Accuracy Top5: 0.8740625
pruning conv14 feature map 889 ...
Loss:  2.0552361726760866 , Accuracy Top1: 0.494375 , Accuracy Top5: 0.86125
pruning conv14 featur

Loss:  1.955592430830002 , Accuracy Top1: 0.5003125 , Accuracy Top5: 0.871875
pruning conv14 feature map 955 ...
Loss:  2.0165903371572496 , Accuracy Top1: 0.5025 , Accuracy Top5: 0.875
pruning conv14 feature map 956 ...
Loss:  1.9874761617183685 , Accuracy Top1: 0.5071875 , Accuracy Top5: 0.86875
pruning conv14 feature map 957 ...
Loss:  2.052974339723587 , Accuracy Top1: 0.4921875 , Accuracy Top5: 0.8609375
pruning conv14 feature map 958 ...
Loss:  1.984663999080658 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.8734375
pruning conv14 feature map 959 ...
Loss:  1.9664281332492828 , Accuracy Top1: 0.513125 , Accuracy Top5: 0.8753125
pruning conv14 feature map 960 ...
Loss:  2.049587960243225 , Accuracy Top1: 0.493125 , Accuracy Top5: 0.864375
pruning conv14 feature map 961 ...
Loss:  2.029774639606476 , Accuracy Top1: 0.49375 , Accuracy Top5: 0.8659375
pruning conv14 feature map 962 ...
Loss:  2.0020281577110293 , Accuracy Top1: 0.4990625 , Accuracy Top5: 0.874375
pruning conv14 feature

Loss:  2.2705630648930866 , Accuracy Top1: 0.34541666666666665 , Accuracy Top5: 0.7238333333333333
prune basing on loss...
fm to be pruned: [  93  333  648  573  344  918  993  693  738  318  108  348  273  764
  963  543  948  153  123  828  183 1023  228   33  798  498  873  168
  468  288   48  888   78  393  513  768  903  633  618  129  783  258
  213  813  423  678   18  708  303  138  603  243  294  723  989  191
  978  378  858 1008  408  558  363  277  759   37  224  483  753  843
  933  438  528   63  453    3  663  579   39  913  588  892  198  617
   44  352  729  954  643  219  644  689  569  924  834  819  442  159
  234  204  549  254 1014   58  831  939  684  339  134  879  899  749
  824  202  329  804  594  457  384  534  832  264  999  472  669  359
  399  849  928  174  624  744  279  389   69  599  794  774  609  524
  479 1019  564  328  494  419  853 1012  427  519  699  414  474  531
 1004  459  464  554  984  929  539   74  854  974  894  324    9  969
  614  5

In [14]:
log=open("Darkent.log",'a+') 
test_pruned_fm(prototxt, caffemodel, 'conv15', 50, 0.375)
log.close()

pruning conv15 feature map 0 ...
Loss:  2.0474978399276735 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.865625
pruning conv15 feature map 1 ...
Loss:  2.048422272205353 , Accuracy Top1: 0.494375 , Accuracy Top5: 0.85875
pruning conv15 feature map 2 ...
Loss:  2.043992040157318 , Accuracy Top1: 0.5 , Accuracy Top5: 0.865
pruning conv15 feature map 3 ...
Loss:  2.025584431886673 , Accuracy Top1: 0.486875 , Accuracy Top5: 0.866875
pruning conv15 feature map 4 ...
Loss:  1.8784162974357606 , Accuracy Top1: 0.52 , Accuracy Top5: 0.8775
pruning conv15 feature map 5 ...
Loss:  2.0990446424484253 , Accuracy Top1: 0.48625 , Accuracy Top5: 0.873125
pruning conv15 feature map 6 ...
Loss:  1.9337528228759766 , Accuracy Top1: 0.51875 , Accuracy Top5: 0.8775
pruning conv15 feature map 7 ...
Loss:  1.9450124073028565 , Accuracy Top1: 0.515625 , Accuracy Top5: 0.87125
pruning conv15 feature map 8 ...
Loss:  2.0403539896011353 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.861875
pruning conv15 feature map

Loss:  2.0444748830795287 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.86625
pruning conv15 feature map 76 ...
Loss:  2.045741045475006 , Accuracy Top1: 0.49375 , Accuracy Top5: 0.85875
pruning conv15 feature map 77 ...
Loss:  2.0300968194007876 , Accuracy Top1: 0.503125 , Accuracy Top5: 0.864375
pruning conv15 feature map 78 ...
Loss:  2.038310273885727 , Accuracy Top1: 0.4825 , Accuracy Top5: 0.866875
pruning conv15 feature map 79 ...
Loss:  1.9031730604171753 , Accuracy Top1: 0.515 , Accuracy Top5: 0.87375
pruning conv15 feature map 80 ...
Loss:  2.1014397239685056 , Accuracy Top1: 0.48625 , Accuracy Top5: 0.8725
pruning conv15 feature map 81 ...
Loss:  1.9254938399791717 , Accuracy Top1: 0.520625 , Accuracy Top5: 0.87875
pruning conv15 feature map 82 ...
Loss:  1.947804045677185 , Accuracy Top1: 0.515625 , Accuracy Top5: 0.871875
pruning conv15 feature map 83 ...
Loss:  2.047804899215698 , Accuracy Top1: 0.495 , Accuracy Top5: 0.86125
pruning conv15 feature map 84 ...
Loss:  2.0682

Loss:  2.0368528032302855 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.86625
pruning conv15 feature map 151 ...
Loss:  2.0131349968910217 , Accuracy Top1: 0.503125 , Accuracy Top5: 0.86375
pruning conv15 feature map 152 ...
Loss:  2.0384600734710694 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.864375
pruning conv15 feature map 153 ...
Loss:  2.0273670077323915 , Accuracy Top1: 0.48625 , Accuracy Top5: 0.868125
pruning conv15 feature map 154 ...
Loss:  1.900407681465149 , Accuracy Top1: 0.515625 , Accuracy Top5: 0.875
pruning conv15 feature map 155 ...
Loss:  2.101684293746948 , Accuracy Top1: 0.4825 , Accuracy Top5: 0.871875
pruning conv15 feature map 156 ...
Loss:  1.9301592481136323 , Accuracy Top1: 0.516875 , Accuracy Top5: 0.8775
pruning conv15 feature map 157 ...
Loss:  1.9550124931335449 , Accuracy Top1: 0.5125 , Accuracy Top5: 0.870625
pruning conv15 feature map 158 ...
Loss:  2.0447850394248963 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.86125
pruning conv15 feature map 159 ...
Lo

Loss:  1.90631560921669 , Accuracy Top1: 0.51875 , Accuracy Top5: 0.879375
pruning conv15 feature map 225 ...
Loss:  2.041400201320648 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.866875
pruning conv15 feature map 226 ...
Loss:  2.0456731152534484 , Accuracy Top1: 0.49375 , Accuracy Top5: 0.85875
pruning conv15 feature map 227 ...
Loss:  2.044472212791443 , Accuracy Top1: 0.49625 , Accuracy Top5: 0.86375
pruning conv15 feature map 228 ...
Loss:  2.0290852618217468 , Accuracy Top1: 0.48875 , Accuracy Top5: 0.866875
pruning conv15 feature map 229 ...
Loss:  1.9020313239097595 , Accuracy Top1: 0.515 , Accuracy Top5: 0.874375
pruning conv15 feature map 230 ...
Loss:  2.107627477645874 , Accuracy Top1: 0.484375 , Accuracy Top5: 0.8725
pruning conv15 feature map 231 ...
Loss:  1.9339382576942443 , Accuracy Top1: 0.5175 , Accuracy Top5: 0.878125
pruning conv15 feature map 232 ...
Loss:  1.9547718453407288 , Accuracy Top1: 0.513125 , Accuracy Top5: 0.87
pruning conv15 feature map 233 ...
Loss:  

Loss:  2.022762575149536 , Accuracy Top1: 0.50625 , Accuracy Top5: 0.869375
pruning conv15 feature map 299 ...
Loss:  1.9072597217559815 , Accuracy Top1: 0.5175 , Accuracy Top5: 0.879375
pruning conv15 feature map 300 ...
Loss:  2.0442707467079164 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.865625
pruning conv15 feature map 301 ...
Loss:  2.047127995491028 , Accuracy Top1: 0.49375 , Accuracy Top5: 0.860625
pruning conv15 feature map 302 ...
Loss:  2.052266387939453 , Accuracy Top1: 0.49875 , Accuracy Top5: 0.8625
pruning conv15 feature map 303 ...
Loss:  2.0316853296756743 , Accuracy Top1: 0.486875 , Accuracy Top5: 0.865625
pruning conv15 feature map 304 ...
Loss:  1.8966734886169434 , Accuracy Top1: 0.511875 , Accuracy Top5: 0.87625
pruning conv15 feature map 305 ...
Loss:  2.098546929359436 , Accuracy Top1: 0.485625 , Accuracy Top5: 0.873125
pruning conv15 feature map 306 ...
Loss:  1.9396077752113343 , Accuracy Top1: 0.5175 , Accuracy Top5: 0.876875
pruning conv15 feature map 307 .

Loss:  1.942147445678711 , Accuracy Top1: 0.50375 , Accuracy Top5: 0.881875
pruning conv15 feature map 373 ...
Loss:  2.0214068341255187 , Accuracy Top1: 0.5075 , Accuracy Top5: 0.866875
pruning conv15 feature map 374 ...
Loss:  1.9013812446594238 , Accuracy Top1: 0.51875 , Accuracy Top5: 0.88
pruning conv15 feature map 375 ...
Loss:  2.050637104511261 , Accuracy Top1: 0.495625 , Accuracy Top5: 0.865625
pruning conv15 feature map 376 ...
Loss:  2.0529449915885927 , Accuracy Top1: 0.489375 , Accuracy Top5: 0.86
pruning conv15 feature map 377 ...
Loss:  2.0516403555870055 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.863125
pruning conv15 feature map 378 ...
Loss:  2.0233602142333984 , Accuracy Top1: 0.486875 , Accuracy Top5: 0.866875
pruning conv15 feature map 379 ...
Loss:  1.899262707233429 , Accuracy Top1: 0.5125 , Accuracy Top5: 0.876875
pruning conv15 feature map 380 ...
Loss:  2.101224946975708 , Accuracy Top1: 0.485625 , Accuracy Top5: 0.87
pruning conv15 feature map 381 ...
Loss:

Loss:  2.0156773829460146 , Accuracy Top1: 0.49125 , Accuracy Top5: 0.865625
pruning conv15 feature map 447 ...
Loss:  1.9450464820861817 , Accuracy Top1: 0.505 , Accuracy Top5: 0.88
pruning conv15 feature map 448 ...
Loss:  2.027353627681732 , Accuracy Top1: 0.505625 , Accuracy Top5: 0.866875
pruning conv15 feature map 449 ...
Loss:  1.8974329555034637 , Accuracy Top1: 0.521875 , Accuracy Top5: 0.8825
pruning conv15 feature map 450 ...
Loss:  2.0368472743034363 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.86375
pruning conv15 feature map 451 ...
Loss:  2.0273092699050905 , Accuracy Top1: 0.49875 , Accuracy Top5: 0.86125
pruning conv15 feature map 452 ...
Loss:  2.0489002203941347 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.863125
pruning conv15 feature map 453 ...
Loss:  2.037256509065628 , Accuracy Top1: 0.486875 , Accuracy Top5: 0.865625
pruning conv15 feature map 454 ...
Loss:  1.890917308330536 , Accuracy Top1: 0.513125 , Accuracy Top5: 0.8775
pruning conv15 feature map 455 ...
Loss:

Loss:  1.5191938773790996 , Accuracy Top1: 0.5290833333333333 , Accuracy Top5: 0.8805833333333334
cut important features...
prune basing on top1...
fm to be pruned: [ 78 155 110 425 333 440 423 350 485 185 230 275 245 170 498  50  65  20
 215 365  35 200  95 140 243 168 305 204 395 288 455 380 320  63 506 290
   5 335 348 468  80 153 483 318 303 363  18 378 453 258 260   3  24 408
   9 410 198 399 123 393 489 213 273 470  33 438 481 249 228 347 108  93
 264 444 138 125 376 500 279 371  69 416 324 174 219 144 311  11 294  84
 189 459  48  54  26 446 281 369 166 114 129 146 159 386 341  41 221  61
 429 436 431 331 286 160  71 421  39 116 361 360 355 339 504 391 356 266
 354 491 248 101 510 100 234 496 251 301 220 206 226 271  76 115 466 461
 161 296 326 310 340 188 384 443   1 236 406 401  91 488 240 316 175  92
  99 309  86  83 176 280 241  45  16 106 121 233 257 128 476 474 210 191
 205 435 490  62  68  15  70 497 505 293 295 203]
Loss:  2.5499870071411133 , Accuracy Top1: 0.3673333333

In [15]:
log=open("Darkent.log",'a+') 
test_pruned_fm(prototxt, caffemodel, 'conv16', 100, 0.375)
log.close()

pruning conv16 feature map 0 ...
Loss:  2.045081629753113 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.86625
pruning conv16 feature map 1 ...
Loss:  2.0464876294136047 , Accuracy Top1: 0.493125 , Accuracy Top5: 0.859375
pruning conv16 feature map 2 ...
Loss:  2.0510065174102783 , Accuracy Top1: 0.49625 , Accuracy Top5: 0.863125
pruning conv16 feature map 3 ...
Loss:  2.023463315963745 , Accuracy Top1: 0.4875 , Accuracy Top5: 0.868125
pruning conv16 feature map 4 ...
Loss:  1.8974513769149781 , Accuracy Top1: 0.51625 , Accuracy Top5: 0.87625
pruning conv16 feature map 5 ...
Loss:  2.091340720653534 , Accuracy Top1: 0.484375 , Accuracy Top5: 0.869375
pruning conv16 feature map 6 ...
Loss:  1.9272448992729188 , Accuracy Top1: 0.520625 , Accuracy Top5: 0.878125
pruning conv16 feature map 7 ...
Loss:  1.9522009682655335 , Accuracy Top1: 0.51375 , Accuracy Top5: 0.870625
pruning conv16 feature map 8 ...
Loss:  2.047272119522095 , Accuracy Top1: 0.495 , Accuracy Top5: 0.8625
pruning conv16 fe

Loss:  2.0463533782958985 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.865625
pruning conv16 feature map 76 ...
Loss:  2.043711397647858 , Accuracy Top1: 0.493125 , Accuracy Top5: 0.858125
pruning conv16 feature map 77 ...
Loss:  2.048067591190338 , Accuracy Top1: 0.49875 , Accuracy Top5: 0.864375
pruning conv16 feature map 78 ...
Loss:  2.0301850748062136 , Accuracy Top1: 0.4875 , Accuracy Top5: 0.86625
pruning conv16 feature map 79 ...
Loss:  1.900632793903351 , Accuracy Top1: 0.515 , Accuracy Top5: 0.87625
pruning conv16 feature map 80 ...
Loss:  2.1006476998329164 , Accuracy Top1: 0.4825 , Accuracy Top5: 0.870625
pruning conv16 feature map 81 ...
Loss:  1.9358774745464324 , Accuracy Top1: 0.5175 , Accuracy Top5: 0.87625
pruning conv16 feature map 82 ...
Loss:  1.9511943364143371 , Accuracy Top1: 0.51375 , Accuracy Top5: 0.870625
pruning conv16 feature map 83 ...
Loss:  2.0487230944633485 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.86125
pruning conv16 feature map 84 ...
Loss:  2.062

Loss:  2.0408714509010313 , Accuracy Top1: 0.49875 , Accuracy Top5: 0.866875
pruning conv16 feature map 151 ...
Loss:  2.0464132380485536 , Accuracy Top1: 0.49375 , Accuracy Top5: 0.86
pruning conv16 feature map 152 ...
Loss:  2.045601437091827 , Accuracy Top1: 0.49875 , Accuracy Top5: 0.86375
pruning conv16 feature map 153 ...
Loss:  2.0248672580718994 , Accuracy Top1: 0.486875 , Accuracy Top5: 0.8675
pruning conv16 feature map 154 ...
Loss:  1.8528485012054443 , Accuracy Top1: 0.519375 , Accuracy Top5: 0.878125
pruning conv16 feature map 155 ...
Loss:  2.0996612334251403 , Accuracy Top1: 0.485625 , Accuracy Top5: 0.873125
pruning conv16 feature map 156 ...
Loss:  1.9314560651779176 , Accuracy Top1: 0.519375 , Accuracy Top5: 0.8775
pruning conv16 feature map 157 ...
Loss:  1.9501019310951233 , Accuracy Top1: 0.515 , Accuracy Top5: 0.87125
pruning conv16 feature map 158 ...
Loss:  2.033864266872406 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.86375
pruning conv16 feature map 159 ...
Loss

Loss:  2.0450418162345887 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.865625
pruning conv16 feature map 226 ...
Loss:  2.030573298931122 , Accuracy Top1: 0.504375 , Accuracy Top5: 0.86375
pruning conv16 feature map 227 ...
Loss:  2.0484914588928222 , Accuracy Top1: 0.49875 , Accuracy Top5: 0.863125
pruning conv16 feature map 228 ...
Loss:  2.0276770102977753 , Accuracy Top1: 0.4875 , Accuracy Top5: 0.8675
pruning conv16 feature map 229 ...
Loss:  1.9003611087799073 , Accuracy Top1: 0.514375 , Accuracy Top5: 0.875
pruning conv16 feature map 230 ...
Loss:  2.0941466021537782 , Accuracy Top1: 0.48375 , Accuracy Top5: 0.871875
pruning conv16 feature map 231 ...
Loss:  1.9300893926620484 , Accuracy Top1: 0.51875 , Accuracy Top5: 0.878125
pruning conv16 feature map 232 ...
Loss:  1.9518991303443909 , Accuracy Top1: 0.514375 , Accuracy Top5: 0.87125
pruning conv16 feature map 233 ...
Loss:  2.045170886516571 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.861875
pruning conv16 feature map 234 ...
L

Loss:  1.9052157163619996 , Accuracy Top1: 0.52 , Accuracy Top5: 0.88125
pruning conv16 feature map 300 ...
Loss:  2.039340033531189 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.86625
pruning conv16 feature map 301 ...
Loss:  2.044404067993164 , Accuracy Top1: 0.49375 , Accuracy Top5: 0.85875
pruning conv16 feature map 302 ...
Loss:  2.0468988609313965 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.86375
pruning conv16 feature map 303 ...
Loss:  2.023893404006958 , Accuracy Top1: 0.48625 , Accuracy Top5: 0.865
pruning conv16 feature map 304 ...
Loss:  1.9016243648529052 , Accuracy Top1: 0.515 , Accuracy Top5: 0.875625
pruning conv16 feature map 305 ...
Loss:  2.101575767993927 , Accuracy Top1: 0.486875 , Accuracy Top5: 0.8725
pruning conv16 feature map 306 ...
Loss:  1.93000767827034 , Accuracy Top1: 0.52125 , Accuracy Top5: 0.878125
pruning conv16 feature map 307 ...
Loss:  1.9547818088531494 , Accuracy Top1: 0.51375 , Accuracy Top5: 0.870625
pruning conv16 feature map 308 ...
Loss:  2.

...

Loss:  2.019419252872467 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.8625
pruning conv16 feature map 746 ...
Loss:  2.0186495566368103 , Accuracy Top1: 0.493125 , Accuracy Top5: 0.865625
pruning conv16 feature map 747 ...
Loss:  1.9487721347808837 , Accuracy Top1: 0.504375 , Accuracy Top5: 0.88
pruning conv16 feature map 748 ...
Loss:  2.0298854899406433 , Accuracy Top1: 0.504375 , Accuracy Top5: 0.8675
pruning conv16 feature map 749 ...
Loss:  1.9045741665363312 , Accuracy Top1: 0.51875 , Accuracy Top5: 0.879375
pruning conv16 feature map 750 ...
Loss:  2.042204005718231 , Accuracy Top1: 0.49625 , Accuracy Top5: 0.865
pruning conv16 feature map 751 ...
Loss:  2.045487265586853 , Accuracy Top1: 0.493125 , Accuracy Top5: 0.85875
pruning conv16 feature map 752 ...
Loss:  2.046680393218994 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.863125
pruning conv16 feature map 753 ...
Loss:  2.026315984725952 , Accuracy Top1: 0.489375 , Accuracy Top5: 0.86625
pruning conv16 feature map 754 ...
Loss:

Loss:  2.065393478870392 , Accuracy Top1: 0.49125 , Accuracy Top5: 0.859375
pruning conv16 feature map 820 ...
Loss:  2.025819470882416 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.86125
pruning conv16 feature map 821 ...
Loss:  2.0194599843025207 , Accuracy Top1: 0.4925 , Accuracy Top5: 0.86625
pruning conv16 feature map 822 ...
Loss:  1.9460930228233337 , Accuracy Top1: 0.503125 , Accuracy Top5: 0.881875
pruning conv16 feature map 823 ...
Loss:  2.0282174587249755 , Accuracy Top1: 0.5075 , Accuracy Top5: 0.8675
pruning conv16 feature map 824 ...
Loss:  1.9051834011077882 , Accuracy Top1: 0.518125 , Accuracy Top5: 0.88125
pruning conv16 feature map 825 ...
Loss:  2.044754936695099 , Accuracy Top1: 0.49625 , Accuracy Top5: 0.86625
pruning conv16 feature map 826 ...
Loss:  2.0447548460960387 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.85875
pruning conv16 feature map 827 ...
Loss:  2.0428973245620727 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.864375
pruning conv16 feature map 828 ...
L

Loss:  2.0469046354293825 , Accuracy Top1: 0.499375 , Accuracy Top5: 0.863125
pruning conv16 feature map 894 ...
Loss:  2.0616524052619933 , Accuracy Top1: 0.491875 , Accuracy Top5: 0.86125
pruning conv16 feature map 895 ...
Loss:  2.026998224258423 , Accuracy Top1: 0.49625 , Accuracy Top5: 0.86125
pruning conv16 feature map 896 ...
Loss:  2.019343478679657 , Accuracy Top1: 0.491875 , Accuracy Top5: 0.86625
pruning conv16 feature map 897 ...
Loss:  1.9407757759094237 , Accuracy Top1: 0.505 , Accuracy Top5: 0.880625
pruning conv16 feature map 898 ...
Loss:  2.0310811614990234 , Accuracy Top1: 0.505625 , Accuracy Top5: 0.866875
pruning conv16 feature map 899 ...
Loss:  1.9064874291419982 , Accuracy Top1: 0.5175 , Accuracy Top5: 0.880625
pruning conv16 feature map 900 ...
Loss:  2.0447262716293335 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.865625
pruning conv16 feature map 901 ...
Loss:  2.0371143770217897 , Accuracy Top1: 0.495625 , Accuracy Top5: 0.86
pruning conv16 feature map 902 ..

Loss:  1.9530242824554442 , Accuracy Top1: 0.51375 , Accuracy Top5: 0.87125
pruning conv16 feature map 968 ...
Loss:  2.0446929049491884 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.861875
pruning conv16 feature map 969 ...
Loss:  2.0660704803466796 , Accuracy Top1: 0.491875 , Accuracy Top5: 0.85875
pruning conv16 feature map 970 ...
Loss:  2.0236619782447813 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.86125
pruning conv16 feature map 971 ...
Loss:  2.0190623021125793 , Accuracy Top1: 0.4925 , Accuracy Top5: 0.86625
pruning conv16 feature map 972 ...
Loss:  1.9368614554405212 , Accuracy Top1: 0.505 , Accuracy Top5: 0.883125
pruning conv16 feature map 973 ...
Loss:  2.021920766830444 , Accuracy Top1: 0.506875 , Accuracy Top5: 0.86625
pruning conv16 feature map 974 ...
Loss:  1.90869882106781 , Accuracy Top1: 0.51875 , Accuracy Top5: 0.88
pruning conv16 feature map 975 ...
Loss:  2.0459965562820432 , Accuracy Top1: 0.49625 , Accuracy Top5: 0.86625
pruning conv16 feature map 976 ...
Loss: 

Loss:  1.7834087592760721 , Accuracy Top1: 0.4835 , Accuracy Top5: 0.85
prune basing on loss...
fm to be pruned: [ 154  104  724  529   34   74  524    4   89   19  764  409  439  599
  349  794  514  719  499  544  364  904  424  649   59  484  589  664
  229  694  184   79  169  919  979  109  334  139  559  889  244 1004
   94  964  949  769  224  859  454  574  604  994  124   64  304  829
  709  274  934 1009  119  239  619  539  269  259  799  509  634  644
  754  389  554  569  289  194   49  394  404  674  284  989  419  214
  374  704  839  314  164  614  739  874  479  944  679  749  854  254
  359  134  449  689  149  379  814  869  824  299  179   14 1019  584
  434  779  329  344  494  899  464  809  844  959  629  199   29  209
  469   44  659  914  784  929  884  974  319  734  786  756  486  792
  741  201  456   57  471    6  936  981  531  576  546  621  276   36
   21  711  306  891  231  801  126  951  351  831  861  636  651  561
  291  216  156  171  876  411 1011

In [16]:
log=open("Darkent.log",'a+') 
test_pruned_fm(prototxt, caffemodel, 'conv17', 100, 0.375)
test_pruned_fm(prototxt, caffemodel, 'conv18', 100, 0.375)
log.close()

pruning conv17 feature map 0 ...
Loss:  2.043882132768631 , Accuracy Top1: 0.4946875 , Accuracy Top5: 0.8625
pruning conv17 feature map 1 ...
Loss:  2.0342166393995287 , Accuracy Top1: 0.495 , Accuracy Top5: 0.865
pruning conv17 feature map 2 ...
Loss:  1.9897192299365998 , Accuracy Top1: 0.50125 , Accuracy Top5: 0.8746875
pruning conv17 feature map 3 ...
Loss:  1.906634345650673 , Accuracy Top1: 0.519375 , Accuracy Top5: 0.8759375
pruning conv17 feature map 4 ...
Loss:  2.053779547214508 , Accuracy Top1: 0.49375 , Accuracy Top5: 0.861875
pruning conv17 feature map 5 ...
Loss:  2.022178139090538 , Accuracy Top1: 0.493125 , Accuracy Top5: 0.86375
pruning conv17 feature map 6 ...
Loss:  1.9835193181037902 , Accuracy Top1: 0.504375 , Accuracy Top5: 0.875625
pruning conv17 feature map 7 ...
Loss:  1.9696133011579513 , Accuracy Top1: 0.51 , Accuracy Top5: 0.8734375
pruning conv17 feature map 8 ...
Loss:  2.0394503116607665 , Accuracy Top1: 0.495625 , Accuracy Top5: 0.8640625
pruning conv17 

Loss:  1.966394145488739 , Accuracy Top1: 0.513125 , Accuracy Top5: 0.8746875
pruning conv17 feature map 75 ...
Loss:  2.047672703266144 , Accuracy Top1: 0.4946875 , Accuracy Top5: 0.861875
pruning conv17 feature map 76 ...
Loss:  2.0302304953336714 , Accuracy Top1: 0.494375 , Accuracy Top5: 0.865625
pruning conv17 feature map 77 ...
Loss:  2.001396372318268 , Accuracy Top1: 0.5009375 , Accuracy Top5: 0.8740625
pruning conv17 feature map 78 ...
Loss:  1.9392139732837677 , Accuracy Top1: 0.516875 , Accuracy Top5: 0.874375
pruning conv17 feature map 79 ...
Loss:  2.049543631076813 , Accuracy Top1: 0.4928125 , Accuracy Top5: 0.8615625
pruning conv17 feature map 80 ...
Loss:  2.021322839856148 , Accuracy Top1: 0.49375 , Accuracy Top5: 0.8634375
pruning conv17 feature map 81 ...
Loss:  1.9837427866458892 , Accuracy Top1: 0.505625 , Accuracy Top5: 0.87375
pruning conv17 feature map 82 ...
Loss:  1.9736703687906265 , Accuracy Top1: 0.508125 , Accuracy Top5: 0.8725
pruning conv17 feature map 8

Loss:  1.9751673793792726 , Accuracy Top1: 0.4990625 , Accuracy Top5: 0.8746875
pruning conv17 feature map 149 ...
Loss:  1.9732088738679885 , Accuracy Top1: 0.510625 , Accuracy Top5: 0.8740625
pruning conv17 feature map 150 ...
Loss:  2.0379874217510223 , Accuracy Top1: 0.4965625 , Accuracy Top5: 0.861875
pruning conv17 feature map 151 ...
Loss:  2.0385722690820693 , Accuracy Top1: 0.491875 , Accuracy Top5: 0.8659375
pruning conv17 feature map 152 ...
Loss:  1.9920474648475648 , Accuracy Top1: 0.5009375 , Accuracy Top5: 0.8740625
pruning conv17 feature map 153 ...
Loss:  1.9474078404903412 , Accuracy Top1: 0.5140625 , Accuracy Top5: 0.8721875
pruning conv17 feature map 154 ...
Loss:  2.0634606969356537 , Accuracy Top1: 0.4915625 , Accuracy Top5: 0.8609375
pruning conv17 feature map 155 ...
Loss:  2.013798765540123 , Accuracy Top1: 0.495 , Accuracy Top5: 0.8640625
pruning conv17 feature map 156 ...
Loss:  1.984901294708252 , Accuracy Top1: 0.5053125 , Accuracy Top5: 0.873125
pruning co

Loss:  1.9995730233192444 , Accuracy Top1: 0.505625 , Accuracy Top5: 0.8659375
pruning conv17 feature map 222 ...
Loss:  2.043477038145065 , Accuracy Top1: 0.49375 , Accuracy Top5: 0.8609375
pruning conv17 feature map 223 ...
Loss:  1.9801366758346557 , Accuracy Top1: 0.4990625 , Accuracy Top5: 0.873125
pruning conv17 feature map 224 ...
Loss:  1.980023410320282 , Accuracy Top1: 0.50625 , Accuracy Top5: 0.871875
pruning conv17 feature map 225 ...
Loss:  2.0395255398750307 , Accuracy Top1: 0.495625 , Accuracy Top5: 0.863125
pruning conv17 feature map 226 ...
Loss:  2.0433773559331896 , Accuracy Top1: 0.4915625 , Accuracy Top5: 0.865
pruning conv17 feature map 227 ...
Loss:  2.000724115371704 , Accuracy Top1: 0.5 , Accuracy Top5: 0.8740625
pruning conv17 feature map 228 ...
Loss:  1.9410528963804246 , Accuracy Top1: 0.5159375 , Accuracy Top5: 0.8740625
pruning conv17 feature map 229 ...
Loss:  2.0774982833862303 , Accuracy Top1: 0.4878125 , Accuracy Top5: 0.859375
pruning conv17 feature 

Loss:  1.9505428338050843 , Accuracy Top1: 0.50375 , Accuracy Top5: 0.8703125
pruning conv17 feature map 295 ...
Loss:  2.0154054141044617 , Accuracy Top1: 0.5021875 , Accuracy Top5: 0.875
pruning conv17 feature map 296 ...
Loss:  1.9867156529426575 , Accuracy Top1: 0.506875 , Accuracy Top5: 0.865625
pruning conv17 feature map 297 ...
Loss:  2.0455659186840056 , Accuracy Top1: 0.4940625 , Accuracy Top5: 0.86
pruning conv17 feature map 298 ...
Loss:  1.9645119494199752 , Accuracy Top1: 0.5009375 , Accuracy Top5: 0.875
pruning conv17 feature map 299 ...
Loss:  1.9661909544467926 , Accuracy Top1: 0.5121875 , Accuracy Top5: 0.8746875
pruning conv17 feature map 300 ...
Loss:  2.0414412069320678 , Accuracy Top1: 0.495625 , Accuracy Top5: 0.863125
pruning conv17 feature map 301 ...
Loss:  2.039209384918213 , Accuracy Top1: 0.4915625 , Accuracy Top5: 0.865
pruning conv17 feature map 302 ...
Loss:  1.999378889799118 , Accuracy Top1: 0.5 , Accuracy Top5: 0.8740625
pruning conv17 feature map 303 

Loss:  2.050686709880829 , Accuracy Top1: 0.494375 , Accuracy Top5: 0.8609375
pruning conv17 feature map 369 ...
Loss:  1.9648702538013458 , Accuracy Top1: 0.5009375 , Accuracy Top5: 0.8709375
pruning conv17 feature map 370 ...
Loss:  2.0124353158473967 , Accuracy Top1: 0.501875 , Accuracy Top5: 0.8734375
pruning conv17 feature map 371 ...
Loss:  1.9993036389350891 , Accuracy Top1: 0.505625 , Accuracy Top5: 0.8671875
pruning conv17 feature map 372 ...
Loss:  2.043957920074463 , Accuracy Top1: 0.4909375 , Accuracy Top5: 0.858125
pruning conv17 feature map 373 ...
Loss:  1.9845527005195618 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.8740625
pruning conv17 feature map 374 ...
Loss:  1.9662258905172347 , Accuracy Top1: 0.5125 , Accuracy Top5: 0.874375
pruning conv17 feature map 375 ...
Loss:  2.0438225722312926 , Accuracy Top1: 0.4953125 , Accuracy Top5: 0.863125
pruning conv17 feature map 376 ...
Loss:  2.0375912588834764 , Accuracy Top1: 0.491875 , Accuracy Top5: 0.865625
pruning conv17 f

Loss:  1.985486056804657 , Accuracy Top1: 0.505 , Accuracy Top5: 0.874375
pruning conv17 feature map 442 ...
Loss:  1.9806272852420808 , Accuracy Top1: 0.5071875 , Accuracy Top5: 0.8728125
pruning conv17 feature map 443 ...
Loss:  2.0311209452152252 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.8615625
pruning conv17 feature map 444 ...
Loss:  1.9624021744728088 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.86875
pruning conv17 feature map 445 ...
Loss:  2.0151276844739914 , Accuracy Top1: 0.5025 , Accuracy Top5: 0.8746875
pruning conv17 feature map 446 ...
Loss:  1.9996455824375152 , Accuracy Top1: 0.5059375 , Accuracy Top5: 0.8665625
pruning conv17 feature map 447 ...
Loss:  2.0425034940242766 , Accuracy Top1: 0.493125 , Accuracy Top5: 0.86
pruning conv17 feature map 448 ...
Loss:  1.9813794720172881 , Accuracy Top1: 0.499375 , Accuracy Top5: 0.87375
pruning conv17 feature map 449 ...
Loss:  1.9626928436756135 , Accuracy Top1: 0.5125 , Accuracy Top5: 0.8753125
pruning conv17 feature ma

Loss:  1.5233843992551168 , Accuracy Top1: 0.53875 , Accuracy Top5: 0.8739166666666667
prune basing on loss...
fm to be pruned: [  3 183 104 348 213  18 123 198 243 468 288  48 453  78 318  33 108 138
  93 258 498 168 333 273 228 483  63 363 378 408 438 423 393 509 153 303
 262 294 119 404 419 219 431 114 494 134 157 129 464 174 479 164 399 314
 204 189 414  14 444  54 384 389 449   9 269 284  84 159 279 354 239 216
 504 324 264  29 234 298 179 434 489  39 369 474 144  89 359 459 339  59
 249 194 309 209  44 299 374  74 254 329  69  99 429  24 337 112   7 493
 276 433 344  17 427  13 292 307  67 382  22 352 247 202  52 149 217 127
 412  97  82 367 472 502 403 232 172  37 487 277 148  21 142 397 141 457
 322  58  73  83 396 351 133 388 246  88 478  51 426 224 223 193 442 268
 208 118 103 508 448 463 238 163 186 253 486 306 321  66  43 328 178  28
 231 497 358   6  36 418  92  81 187 373 291 126]
Loss:  1.5539864913622539 , Accuracy Top1: 0.5306666666666666 , Accuracy Top5: 0.86575
cut i

In [ ]:
log=open("Darkent.log",'a+') 
test_pruned_fm(prototxt, caffemodel, 'conv14', 50, 0.25)
log.close()

pruning conv14 feature map 0 ...
Loss:  2.0398403143882753 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.865625
pruning conv14 feature map 1 ...
Loss:  2.0534322381019594 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.86
pruning conv14 feature map 2 ...
Loss:  2.0428766894340513 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.86375
pruning conv14 feature map 3 ...
Loss:  2.034882365465164 , Accuracy Top1: 0.485 , Accuracy Top5: 0.86625
pruning conv14 feature map 4 ...
Loss:  1.903494210243225 , Accuracy Top1: 0.5125 , Accuracy Top5: 0.8775
pruning conv14 feature map 5 ...
Loss:  2.0984668922424317 , Accuracy Top1: 0.485625 , Accuracy Top5: 0.871875
pruning conv14 feature map 6 ...
Loss:  1.932931661605835 , Accuracy Top1: 0.518125 , Accuracy Top5: 0.878125
pruning conv14 feature map 7 ...
Loss:  1.9598223161697388 , Accuracy Top1: 0.513125 , Accuracy Top5: 0.87
pruning conv14 feature map 8 ...
Loss:  2.0385415363311767 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.861875
pruning conv14 featur

Loss:  2.036237268447876 , Accuracy Top1: 0.496875 , Accuracy Top5: 0.865
pruning conv14 feature map 76 ...
Loss:  2.030468761920929 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.86125
pruning conv14 feature map 77 ...
Loss:  2.0490131783485412 , Accuracy Top1: 0.49625 , Accuracy Top5: 0.863125
pruning conv14 feature map 78 ...
Loss:  2.0265704667568207 , Accuracy Top1: 0.485625 , Accuracy Top5: 0.865625
pruning conv14 feature map 79 ...
Loss:  1.9068521213531495 , Accuracy Top1: 0.51125 , Accuracy Top5: 0.875625
pruning conv14 feature map 80 ...
Loss:  2.100471947193146 , Accuracy Top1: 0.485 , Accuracy Top5: 0.8725
pruning conv14 feature map 81 ...
Loss:  1.9322860300540925 , Accuracy Top1: 0.519375 , Accuracy Top5: 0.878125
pruning conv14 feature map 82 ...
Loss:  1.9440723514556886 , Accuracy Top1: 0.515 , Accuracy Top5: 0.869375
pruning conv14 feature map 83 ...
Loss:  2.03280885219574 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.8625
pruning conv14 feature map 84 ...
Loss:  2.066577

Loss:  2.035093092918396 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.86625
pruning conv14 feature map 151 ...
Loss:  2.031903429031372 , Accuracy Top1: 0.489375 , Accuracy Top5: 0.861875
pruning conv14 feature map 152 ...
Loss:  2.0522410249710084 , Accuracy Top1: 0.4925 , Accuracy Top5: 0.863125
pruning conv14 feature map 153 ...
Loss:  2.0254599928855894 , Accuracy Top1: 0.4875 , Accuracy Top5: 0.868125
pruning conv14 feature map 154 ...
Loss:  1.8860685563087463 , Accuracy Top1: 0.51875 , Accuracy Top5: 0.878125
pruning conv14 feature map 155 ...
Loss:  2.1082972502708435 , Accuracy Top1: 0.483125 , Accuracy Top5: 0.868125
pruning conv14 feature map 156 ...
Loss:  1.9325037157535554 , Accuracy Top1: 0.519375 , Accuracy Top5: 0.8775
pruning conv14 feature map 157 ...
Loss:  1.9468257379531861 , Accuracy Top1: 0.515625 , Accuracy Top5: 0.87
pruning conv14 feature map 158 ...
Loss:  2.0305484890937806 , Accuracy Top1: 0.5 , Accuracy Top5: 0.863125
pruning conv14 feature map 159 ...
Lo

Loss:  2.041511471271515 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.86625
pruning conv14 feature map 226 ...
Loss:  2.0536378383636475 , Accuracy Top1: 0.4925 , Accuracy Top5: 0.858125
pruning conv14 feature map 227 ...
Loss:  2.0507382583618163 , Accuracy Top1: 0.49875 , Accuracy Top5: 0.863125
pruning conv14 feature map 228 ...
Loss:  2.025551174879074 , Accuracy Top1: 0.48875 , Accuracy Top5: 0.8675
pruning conv14 feature map 229 ...
Loss:  1.9027050709724427 , Accuracy Top1: 0.514375 , Accuracy Top5: 0.87625
pruning conv14 feature map 230 ...
Loss:  2.1015685629844665 , Accuracy Top1: 0.484375 , Accuracy Top5: 0.8725
pruning conv14 feature map 231 ...
Loss:  1.9346095073223113 , Accuracy Top1: 0.518125 , Accuracy Top5: 0.87875
pruning conv14 feature map 232 ...
Loss:  1.9522830319404603 , Accuracy Top1: 0.515 , Accuracy Top5: 0.870625
pruning conv14 feature map 233 ...
Loss:  2.0388904428482055 , Accuracy Top1: 0.495625 , Accuracy Top5: 0.86125
pruning conv14 feature map 234 ...
Lo

Loss:  1.912303992509842 , Accuracy Top1: 0.516875 , Accuracy Top5: 0.879375
pruning conv14 feature map 300 ...
Loss:  2.043667480945587 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.86625
pruning conv14 feature map 301 ...
Loss:  2.026132402420044 , Accuracy Top1: 0.498125 , Accuracy Top5: 0.863125
pruning conv14 feature map 302 ...
Loss:  2.048621633052826 , Accuracy Top1: 0.4975 , Accuracy Top5: 0.86375
pruning conv14 feature map 303 ...
Loss:  2.030402866601944 , Accuracy Top1: 0.4875 , Accuracy Top5: 0.865625
pruning conv14 feature map 304 ...
Loss:  1.9054902768135071 , Accuracy Top1: 0.51375 , Accuracy Top5: 0.87625
pruning conv14 feature map 305 ...
Loss:  2.092039580345154 , Accuracy Top1: 0.48375 , Accuracy Top5: 0.86875
pruning conv14 feature map 306 ...
Loss:  1.9367967069149017 , Accuracy Top1: 0.5175 , Accuracy Top5: 0.87625
pruning conv14 feature map 307 ...
Loss:  1.9468219566345215 , Accuracy Top1: 0.508125 , Accuracy Top5: 0.868125
pruning conv14 feature map 308 ...
Lo

In [8]:
import caffe
import numpy as np
from numpy import array
from numpy.random import normal
from matplotlib import pyplot
import os, sys
import struct
prototxt = '../Darknet19/Darknet19.prototxt'
caffemodel = '../Darknet19/Darknet19.caffemodel'
TEST_ITER = 375
caffe.set_device(0)  
caffe.set_mode_gpu()


In [9]:
aa_index=np.zeros(32)
aa_index=[6,14,31,13,11,1,25,15,9,0,21,5,8,26,24,30,29,17,27,2,19,16,18,10,4,20,3,23,28,12,7,22]

def test_accuracy(net, test_iter):
    loss = np.zeros(1)
    top1 = np.zeros(1)
    top5 = np.zeros(1)
    test_loss = 0
    top1_accuracy = 0
    top5_accuracy = 0
    for test_it in range(test_iter):
        net.forward()
        test_loss += net.blobs['loss'].data
        top1_accuracy += net.blobs['accuracy'].data  
        top5_accuracy += net.blobs['accuracy-top5'].data
    loss[0] = test_loss / test_iter 
    top1[0] = top1_accuracy / test_iter
    top5[0] = top5_accuracy / test_iter
    print 'Loss: ', loss[0], ', Accuracy Top1:', top1[0], ', Accuracy Top5:', top5[0]
    return loss[0], top1[0], top5[0]

layer='conv1'

zeros=np.zeros(net.params[layer][0].data[0].shape)
net = caffe.Net(prototxt, caffemodel, caffe.TEST)
test_accuracy(net, 375)

for i in range(12):
    print aa_index[i]
    net = caffe.Net(prototxt, caffemodel, caffe.TEST)
    net.params[layer][0].data[aa_index[i]]=zeros

test_accuracy(net, 375)

Loss:  0.7005859552621841 , Accuracy Top1: 0.796 , Accuracy Top5: 0.9673333333333334
6
14
31
13
11
1
25
15
9
0
21
5
Loss:  0.7194335547685623 , Accuracy Top1: 0.7888333333333334 , Accuracy Top5: 0.96625


(0.7194335547685623, 0.7888333333333334, 0.96625)